In [ ]:
!pip install BisPy
!pip install ogb
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for llist: filename=llist-0.7.1-cp310-cp310-linux_x86_64.whl size=74083 sha256=855c519471cd92dbdf3715c8f265ec631f4562c49b6eab842d90119d4775d9b5
  Stored in directory: /root/.cache/pip/wheels/e9/91/41/252becb26acf296e14913e0ee13940295bc475be57aed9bb2e
Successfully built llist
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7026 sha256=3a489dd19bcbd6e4d7a0c893eed849bbd1862f53d3f9846bb34a52fa9631933a
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built littleutils
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.3 MB/s eta 0:00:00


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import random
import numpy as np
import torch
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
import pandas as pd
from torch_geometric.datasets import Planetoid, CoraFull, Yelp, WebKB, Actor
import os
import bispy
from bispy import compute_maximum_bisimulation, Algorithms
from bispy.dovier_piazza_policriti.dovier_piazza_policriti import dovier_piazza_policriti_partition
from bispy.dovier_piazza_policriti.ranked_partition import RankedPartition
from bispy.dovier_piazza_policriti.dovier_piazza_policriti import dovier_piazza_policriti, collapse, build_block_counterimage, split_upper_ranks
from bispy.utilities.rank_computation import compute_rank
from bispy.utilities.graph_decorator import decorate_nx_graph, decorate_bispy_graph,as_bispy_graph,to_tuple_list
from bispy.utilities import *
import numpy as np
from sklearn.cluster import KMeans
from collections import Counter
import random
import copy
import time
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
import matplotlib.pyplot as plt
import numpy as np
from ogb.nodeproppred import PygNodePropPredDataset
import numpy as np
import torch
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
from sklearn.metrics.pairwise import cosine_similarity

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
MAG240M_Syn = torch.load('gdrive/My Drive/GNN_Sparsification/MAG240M_synthetic_ws_graph_30core_6_0.5_directed.pt')
MAG240M_Syn

Data(x=[2000000, 800], edge_index=[2, 5999965], y=[2000000])

In [ ]:
model_save_path_GCN = os.path.join('gdrive/My Drive/GNN_Sparsification/Pre_Trained_MAG_GCN_2_Layers.pt')
model_save_path_GAT = os.path.join('gdrive/My Drive/GNN_Sparsification/Pre_Trained_MAG_GAT_2_Layers.pt')
model_save_path_GraphSage = os.path.join('gdrive/My Drive/GNN_Sparsification/Pre_Trained_MAG_GraphSAGE_2_Layers.pt')

In [ ]:
# Load tensor from .pt file
# loaded_tensor = torch.load('gdrive/My Drive/GNN_Sparsification/k_spanner_100.pt')
# loaded_tensor = torch.load('gdrive/My Drive/GNN_Sparsification/arxiv_sparsified_0.35.pt')
# loaded_tensor = torch.load('gdrive/My Drive/GNN_Sparsification/arxiv_sparsified_1.55.pt')
# loaded_tensor = torch.load('gdrive/My Drive/GNN_Sparsification/Baselines/D_spar_INDUCED/10%/10%_induced_arxiv_sparsified_0.35.pt')
loaded_tensor = torch.load('gdrive/My Drive/GNN_Sparsification/Baselines/K_spanner_INDUCED/10%/2_spanner.pt')
# data.edge_index = loaded_tensor
# data
loaded_tensor


tensor([[    0,     1,     5,  ..., 94770, 94771, 94772],
        [52271, 75193, 34202,  ..., 88480, 75905, 63895]])

In [ ]:
import copy
from torch_geometric.utils import subgraph
train_mask = torch.rand(MAG240M_Syn.num_nodes) < 0.99
test_mask = ~train_mask
tested_elements =  torch.nonzero(test_mask==True).squeeze()
tested_elements = tested_elements.tolist()

# file_path = "gdrive/My Drive/GNN_Sparsification/5%_ogbn_arxiv_train_mask.pt"

# file_path = "gdrive/My Drive/GNN_Sparsification/10%_ogbn_product_train_mask.pt"

file_path = "gdrive/My Drive/GNN_Sparsification/1%_MAG240M_train_mask.pt"

# Save the binary tensor mask to a .pt file
torch.save(train_mask, file_path)

In [ ]:
# train_mask = torch.load('gdrive/My Drive/GNN_Sparsification/10%_ogbn_arxiv_train_mask.pt')
# train_mask = torch.load("gdrive/My Drive/GNN_Sparsification/10%_ogbn_product_train_mask.pt")
# train_mask = torch.load("gdrive/My Drive/GNN_Sparsification/10%_yelp_train_mask.pt")
train_mask = torch.load('gdrive/My Drive/GNN_Sparsification/0.5%_MAG240M_train_mask.pt')
test_mask = ~train_mask
tested_elements =  torch.nonzero(test_mask==True).squeeze()
tested_elements = tested_elements.tolist()

In [ ]:
# Convert PyG graph to NetworkX graph
def pyg_to_networkx(pyg_data):
    edge_index = pyg_data.edge_index
    num_nodes = 2000000
    edge_list = edge_index.t().tolist()
    G = nx.DiGraph()
    G.add_nodes_from(range(num_nodes))
    G.add_edges_from(edge_list)
    return G

networkx_graph = pyg_to_networkx(MAG240M_Syn)

In [ ]:
num_nodes = networkx_graph.number_of_nodes()
num_edges = networkx_graph.number_of_edges()

print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)
partition = compute_maximum_bisimulation(networkx_graph)
len(partition)

Number of nodes: 2000000
Number of edges: 6000000


747660

In [ ]:
#Watts-Strogatz model param
k = 10
p = 0.95
num_nodes = 100000
#Generate Watts-Strogatz graph
ws_graph = nx.watts_strogatz_graph(num_nodes, k, p)
ws_edges = torch.tensor(list(ws_graph.edges), dtype = torch.long).t().contiguous()

G = nx.DiGraph()
G.add_nodes_from(range(num_nodes))
G.add_edges_from(ws_edges.t().tolist())


num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)
partition = compute_maximum_bisimulation(G)
len(partition)

Number of nodes: 100000
Number of edges: 500000


73618

In [ ]:
# Watts-Strogatz model param
# smaller k --> less EC
# larger p --> less EC

k = 5
p = 0.95
num_nodes = 100000
#Generate Watts-Strogatz graph
ws_graph = nx.watts_strogatz_graph(num_nodes, k, p)
ws_edges = torch.tensor(list(ws_graph.edges), dtype = torch.long).t().contiguous()

G = nx.DiGraph()
G.add_nodes_from(range(num_nodes))
G.add_edges_from(ws_edges.t().tolist())


num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)
partition = compute_maximum_bisimulation(G)
len(partition)

Number of nodes: 100000
Number of edges: 200000


20386

In [ ]:
#Watts-Strogatz model param
k = 4
p = 0.75
num_nodes = 100000
#Generate Watts-Strogatz graph
ws_graph = nx.watts_strogatz_graph(num_nodes, k, p)
ws_edges = torch.tensor(list(ws_graph.edges), dtype = torch.long).t().contiguous()

G = nx.DiGraph()
# G.add_nodes_from(range(num_nodes))
G.add_edges_from(ws_edges.t().tolist())


num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)
partition = compute_maximum_bisimulation(G)
len(partition)

Number of nodes: 100000
Number of edges: 200000


46008

In [ ]:
ws_edges[:,100]

tensor([   29, 28166])

In [ ]:
num_nodes = networkx_graph.number_of_nodes()
num_edges = networkx_graph.number_of_edges()

print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

partition = compute_maximum_bisimulation(networkx_graph)
len(partition)

Number of nodes: 2000000
Number of edges: 11999992


1557775

In [ ]:
num_nodes = networkx_graph.number_of_nodes()
num_edges = networkx_graph.number_of_edges()

print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

partition = compute_maximum_bisimulation(networkx_graph, algorithm=Algorithms.DovierPiazzaPolicriti)
len(partition)

Number of nodes: 2000000
Number of edges: 11999992


1557775

In [ ]:
# compute_maximum_bisimulation(graph, algorithm=Algorithms.DovierPiazzaPolicriti)

In [ ]:
for i in range(200):
  print(len(partition[i]))

1204
21312
2057
17939
321
427
1545
2240
7
9
34
36
44
28
190
80
1
1
2
1
1
1
2
17
3
16
104
1
1
63
92
5
68
467
7
2
251
6
1
1
1
1
1
6
2
17
1
1
1
1
3
7
3
10
9
8
1
23
40
69
1
1
1
2
1
4
7
1
1
17
1
1
1
1
1
3
3
1
1
1
1
1
1
1
1
1
1
1
2
2
1
2
4
1
1
1
1
1
2
5
4
1
1
1
1
1
1
1
1
1
2
2
2
4
1
1
2
7
5
7
8
19
2
1
1
1
1
1
1
1
3
1
1
5
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
285
1404
265
1075
71
36
24
4
14
4
1
1
1
1
1
1
58
7
1
14
1
1
1
3
1
1
1
3
8
3
2
5
2
1
1
1


In [ ]:
num_nodes = networkx_graph.number_of_nodes()
num_edges = networkx_graph.number_of_edges()

print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

partition = compute_maximum_bisimulation(networkx_graph)
len(partition)

Number of nodes: 2000000
Number of edges: 11999940


1984969

In [ ]:
# # small_world_graph
# num_nodes = networkx_graph.number_of_nodes()
# num_edges = networkx_graph.number_of_edges()

# print("Number of nodes:", num_nodes)
# print("Number of edges:", num_edges)

partition = compute_maximum_bisimulation(small_world_hierarchy_graph)
len(partition)

24

## Small-world Graph Generator

In [ ]:
import networkx as nx
import random

def generate_small_world_hierarchy(num_nodes, num_ranks, average_degree, rewiring_probability):
    G = nx.DiGraph()

    # Create nodes with ranks
    nodes_per_rank = num_nodes // num_ranks
    for rank in range(num_ranks):
        for i in range(nodes_per_rank):
            node_id = rank * nodes_per_rank + i
            G.add_node(node_id, rank=rank)

    # Connect nodes within the same rank without requiring connectivity
    for rank in range(num_ranks):
        nodes_in_rank = [node for node, data in G.nodes(data=True) if data['rank'] == rank]
        for node in nodes_in_rank:
            for _ in range(average_degree // 2):
                other_node = random.choice(nodes_in_rank)
                G.add_edge(node, other_node)

    # Connect nodes between ranks with rewiring
    for rank in range(num_ranks - 1):
        nodes_in_current_rank = [node for node, data in G.nodes(data=True) if data['rank'] == rank]
        nodes_in_next_rank = [node for node, data in G.nodes(data=True) if data['rank'] == rank + 1]

        for node in nodes_in_current_rank:
            for _ in range(average_degree // 4):
                neighbor = random.choice(nodes_in_next_rank)
                G.add_edge(node, neighbor)

                # Rewiring with probability
                if random.random() < rewiring_probability:
                    new_neighbor = random.choice(nodes_in_next_rank)
                    G.add_edge(node, new_neighbor)

    # Remove all self-loops
    G.remove_edges_from(nx.selfloop_edges(G))

    return G

# Example usage
num_nodes = 10000
num_ranks = 200
average_degree = 2
rewiring_probability = 0.5

small_world_hierarchy_graph = generate_small_world_hierarchy(num_nodes, num_ranks, average_degree, rewiring_probability)


In [ ]:
len(small_world_hierarchy_graph.edges())

9976

# Radius-based K-hops Induced Subgraph

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Define two tensors as NumPy arrays
tensor1 = np.array([1, 2, 3])
tensor2 = np.array([4, 5, 6])

# Reshape the tensors into 2D arrays for cosine_similarity function
tensor1 = tensor1.reshape(1, -1)
tensor2 = tensor2.reshape(1, -1)

# Calculate cosine similarity
similarity = cosine_similarity(data.x[tested_elements[2]].numpy().reshape(1, -1), data.x[tested_elements[1000]].numpy().reshape(1, -1))

print("Cosine Similarity using scikit-learn:", similarity[0][0])

Cosine Similarity using scikit-learn: 0.5655823


In [ ]:
# nodes_to_find_neighbors = tested_elements
# neighbors = set()
# up_to_k_hop_neighbors = []
# up_to_k_hop_neighbors_list = []
# k = 1
# # G = nx.Graph()
# G = nx.DiGraph()

# for i in range(k):
#   # Iterate through the nodes in your set
#   print(i)
#   neighbors = set()
#   for node in nodes_to_find_neighbors:
#     # Use the neighbors() function to get 1-hop neighbors and add them to the set
#     # neighbors.update(networkx_graph.neighbors(node))
#     # neighbors.update(networkx_graph.predecessors(node))

#     if i == 0:
#       G.add_node(node)

#       for neighbor in networkx_graph.neighbors(node):
#         similarity = cosine_similarity(data.x[node].numpy().reshape(1, -1), data.x[neighbor].numpy().reshape(1, -1))
#         if neighbor not in nodes_to_find_neighbors and similarity > 0.5:
#           G.add_edge(node, neighbor)
#           neighbors.update([neighbor])


#       for neighbor in networkx_graph.predecessors(node):
#         similarity = cosine_similarity(data.x[node].numpy().reshape(1, -1), data.x[neighbor].numpy().reshape(1, -1))
#         if neighbor not in nodes_to_find_neighbors and similarity > 0.5:
#           G.add_edge(neighbor, node)
#           neighbors.update([neighbor])


#     else:
#       for neighbor in networkx_graph.neighbors(node):
#         similarity = cosine_similarity(data.x[node].numpy().reshape(1, -1), data.x[neighbor].numpy().reshape(1, -1))
#         if neighbor not in up_to_k_hop_neighbors_list and similarity > 0.5:
#           G.add_edge(node, neighbor)
#           neighbors.update([neighbor])

#       for neighbor in networkx_graph.predecessors(node):
#         similarity = cosine_similarity(data.x[node].numpy().reshape(1, -1), data.x[neighbor].numpy().reshape(1, -1))
#         if neighbor not in up_to_k_hop_neighbors_list and similarity > 0.5:
#           G.add_edge(neighbor, node)
#           neighbors.update([neighbor])




#   # ensure the disjoint property
#   if i > 0:
#     for ele in up_to_k_hop_neighbors:
#       neighbors_list = list(set(list(neighbors)).difference(set(ele)))
#     neighbors_list = list(set(neighbors).difference(set(tested_elements)))

#   if i == 0:
#     neighbors_list = list(set(neighbors).difference(set(tested_elements)))
#     up_to_k_hop_neighbors_list += tested_elements

#   nodes_to_find_neighbors = neighbors_list
#   up_to_k_hop_neighbors.append(neighbors_list)
#   up_to_k_hop_neighbors_list += neighbors_list

## Directed

In [ ]:
nodes_to_find_neighbors = tested_elements
neighbors = set()
up_to_k_hop_neighbors = []
up_to_k_hop_neighbors_list = []
k = 1
# G = nx.Graph()
G = nx.DiGraph()

for i in range(k):
  # Iterate through the nodes in your set
  print(i)
  neighbors = set()
  for node in nodes_to_find_neighbors:
    # Use the neighbors() function to get 1-hop neighbors and add them to the set
    neighbors.update(networkx_graph.neighbors(node))
    neighbors.update(networkx_graph.predecessors(node))

    if i == 0:
      G.add_node(node)

      for neighbor in networkx_graph.neighbors(node):
        G.add_edge(node, neighbor)

      for neighbor in networkx_graph.predecessors(node):
        G.add_edge(neighbor, node)


    else:
      for neighbor in networkx_graph.neighbors(node):
        if neighbor not in up_to_k_hop_neighbors_list:
          G.add_edge(node, neighbor)

      for neighbor in networkx_graph.predecessors(node):
        if neighbor not in up_to_k_hop_neighbors_list:
          G.add_edge(neighbor, node)




  # ensure the disjoint property
  if i > 0:
    for ele in up_to_k_hop_neighbors:
      neighbors_list = list(set(list(neighbors)).difference(set(ele)))
    neighbors_list = list(set(neighbors).difference(set(tested_elements)))

  if i == 0:
    neighbors_list = list(set(neighbors).difference(set(tested_elements)))
    up_to_k_hop_neighbors_list += tested_elements

  nodes_to_find_neighbors = neighbors_list
  up_to_k_hop_neighbors.append(neighbors_list)
  up_to_k_hop_neighbors_list += neighbors_list

0


## Undirected

In [ ]:
nodes_to_find_neighbors = tested_elements
neighbors = set()
up_to_k_hop_neighbors = []
up_to_k_hop_neighbors_list = []
k = 1
G = nx.DiGraph()


for i in range(k):
  # Iterate through the nodes in your set
  print(i)
  neighbors = set()
  for node in nodes_to_find_neighbors:
    # Use the neighbors() function to get 1-hop neighbors and add them to the set
    neighbors.update(networkx_graph.neighbors(node))


    if i == 0:
      G.add_node(node)
      for neighbor in networkx_graph.neighbors(node):
        if neighbor not in nodes_to_find_neighbors:
          G.add_edge(node, neighbor)

    else:
      for neighbor in networkx_graph.neighbors(node):
        if neighbor not in up_to_k_hop_neighbors_list:
          G.add_edge(node, neighbor)




  # ensure the disjoint property
  if i > 0:
    for ele in up_to_k_hop_neighbors:
      neighbors_list = list(set(list(neighbors)).difference(set(ele)))
    neighbors_list = list(set(neighbors).difference(set(tested_elements)))

  if i == 0:
    neighbors_list = list(set(neighbors).difference(set(tested_elements)))
    up_to_k_hop_neighbors_list += tested_elements

  nodes_to_find_neighbors = neighbors_list
  up_to_k_hop_neighbors.append(neighbors_list)
  up_to_k_hop_neighbors_list += neighbors_list

0


In [ ]:
nodes_to_find_neighbors = tested_elements
neighbors = set()
up_to_k_hop_neighbors = []
up_to_k_hop_neighbors_list = []
k = 1
G = nx.DiGraph()


for i in range(k):
  # Iterate through the nodes in your set
  print(i)
  neighbors = set()
  for node in nodes_to_find_neighbors:
    if i == 0:
      G.add_node(node)

    if len(list(networkx_graph.neighbors(node))) > 5:
      num_elements_to_select = 5
      original_list = list(networkx_graph.neighbors(node))
      random_elements = random.sample(original_list, num_elements_to_select)
      new_list = list(random_elements)
      neighbors.update(new_list)
      for neighbor in new_list:
        G.add_edge(node, neighbor)

    else:
      neighbors.update(networkx_graph.neighbors(node))
      for neighbor in networkx_graph.neighbors(node):
        G.add_edge(node, neighbor)







0


In [ ]:
nodes_to_find_neighbors = tested_elements
neighbors = set()
up_to_k_hop_neighbors = []
up_to_k_hop_neighbors_list = []
k = 1
G = nx.DiGraph()


for i in range(k):
  # Iterate through the nodes in your set
  print(i)
  neighbors = set()
  for node in nodes_to_find_neighbors:
    if i == 0:
      G.add_node(node)


    for neighbor in networkx_graph.neighbors(node):
      G.add_edge(node, neighbor)







0


In [ ]:
nodes_to_find_neighbors = tested_elements
neighbors = set()
up_to_k_hop_neighbors = []
up_to_k_hop_neighbors_list = []
k = 2
G = nx.DiGraph()


for i in range(k):
  # Iterate through the nodes in your set
  print(i)
  neighbors = set()
  for node in nodes_to_find_neighbors:
    # Use the neighbors() function to get 1-hop neighbors and add them to the set
    neighbors.update(networkx_graph.predecessors(node))


    if i == 0:
      G.add_node(node)

      for neighbor in networkx_graph.predecessors(node):
        # if neighbor not in nodes_to_find_neighbors:
        G.add_edge(neighbor, node)


    else:
      for neighbor in networkx_graph.predecessors(node):
        if neighbor not in up_to_k_hop_neighbors_list:
        # if neighbor not in nodes_to_find_neighbors:
          G.add_edge(neighbor, node)


  # ensure the disjoint property
  if i > 0:
    for ele in up_to_k_hop_neighbors:
      neighbors_list = list(set(list(neighbors)).difference(set(ele)))
    neighbors_list = list(set(neighbors).difference(set(tested_elements)))


  if i == 0:
    neighbors_list = list(set(neighbors).difference(set(tested_elements)))
    up_to_k_hop_neighbors_list += tested_elements


  nodes_to_find_neighbors = neighbors_list
  up_to_k_hop_neighbors.append(neighbors_list)
  up_to_k_hop_neighbors_list += neighbors_list

0
1


In [ ]:
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

partition = compute_maximum_bisimulation(G)
len(partition)

Number of nodes: 68080
Number of edges: 59116


36

In [ ]:
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

partition = compute_maximum_bisimulation(G)
len(partition)

Number of nodes: 94773
Number of edges: 221911


28292

In [ ]:
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

partition = compute_maximum_bisimulation(G)
len(partition)

Number of nodes: 115255
Number of edges: 340335


45696

In [ ]:
import pickle
# Assuming you have a NetworkX graph called 'graph'
with open("gdrive/My Drive/GNN_Sparsification/10%_Induced_SubG_Yelp.pkl", "wb") as f:
    pickle.dump(G, f)

In [ ]:
import pickle
# Assuming you have a NetworkX graph called 'graph'
with open("gdrive/My Drive/GNN_Sparsification/10%_Induced_SubG_Ogbn_Product.pkl", "wb") as f:
    pickle.dump(G, f)

In [ ]:
import pickle
# Assuming you have a NetworkX graph called 'graph'
with open("gdrive/My Drive/GNN_Sparsification/15%_Induced_SubG_Ogbn_Arxiv.pkl", "wb") as f:
    pickle.dump(G, f)

In [ ]:
def find_tuple_index_with_integer(list_of_tuples, integer):
    return next((index for index, tup in enumerate(list_of_tuples) if integer in tup), -1)

In [ ]:
# Function to split long tuples
def split_long_tuples(lst, max_length):
    new_list = []
    for tup in lst:
        if len(tup) > max_length:
            # Split the tuple into smaller tuples
            split_tuples = [tup[i:i+max_length] for i in range(0, len(tup), max_length)]
            new_list.extend(split_tuples)
        else:
            new_list.append(tup)
    return new_list

# Split tuples with length over 10K into tuples with length 10K or under
max_length = 500
partition = split_long_tuples(partition, max_length)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import euclidean
from scipy.cluster.hierarchy import linkage, fcluster
import numpy as np

def exclude_tested_elements_from_MB(parition, tested_elements):
  tuple_list = partition

  # Sort the list by the number of elements in each tuple
  sorted_list = sorted(tuple_list, key=lambda x: len(x), reverse=True)

  indices = []
  list_of_lists = [list(t) for t in sorted_list]

  for ele in tested_elements:
    id = find_tuple_index_with_integer(sorted_list, ele)
    if len(list_of_lists[id]) > 1:
      list_of_lists[id].remove(ele)
      list_of_lists.append([ele])
      indices.append(len(list_of_lists)-1)
    else:
      indices.append(id)

  return list_of_lists, indices

def alpha_Sparsification(list_of_lists, data, alpha):
  list_of_tuples = [tuple(inner_list) for inner_list in list_of_lists]
  i = 0
  while len(list_of_lists[i])>1:

      Z = linkage(data.x[list_of_tuples[i],:], method='average', metric='cosine')
      threshold = alpha * np.max(Z[:, 2])
      clusters = fcluster(Z, threshold, criterion='distance')

      # Get cluster labels for each data point
      labels = clusters


      # Z = linkage(data.x[list_of_tuples[i],:], method='average', metric='euclidean')
      # threshold = alpha * np.max(Z[:, 2])
      # clusters = fcluster(Z, threshold, criterion='distance')

      # # Get cluster labels for each data point
      # labels = clusters




      # Sample tuple to split
      data_tuple = list_of_tuples[i]

      # Create a dictionary to store split data
      split_data = {}

      # Iterate through the labels and tuple elements
      for label, value in zip(labels, data_tuple):
        if label not in split_data:
          split_data[label] = []
        split_data[label].append(value)

      # Convert the dictionary to a list of tuples (if needed)
      split_tuples = [tuple(values) for key, values in split_data.items()]
      list_of_tuples.pop(i)
      j = 0
      for values in split_tuples:
        if j == 0:
          list_of_tuples.insert(0, values)
        else:
          list_of_tuples.insert(len(list_of_tuples), values)
        j+=1

      i+=1

  result_dict = {element: index for index, tup in enumerate(list_of_tuples) for element in tup}

  return list_of_tuples, result_dict

In [ ]:
# Function to split long tuples
def split_long_tuples(lst, max_length):
    new_list = []
    for tup in lst:
        if len(tup) > max_length:
            # Split the tuple into smaller tuples
            split_tuples = [tup[i:i+max_length] for i in range(0, len(tup), max_length)]
            new_list.extend(split_tuples)
        else:
            new_list.append(tup)
    return new_list

# Split tuples with length over 10K into tuples with length 10K or under
max_length = 500
partition = split_long_tuples(partition, max_length)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import euclidean
from scipy.cluster.hierarchy import linkage, fcluster
import numpy as np

def exclude_tested_elements_from_MB(partition, tested_elements):
    tuple_dict = {}
    indices = []

    tuple_list = partition

    # Sort the list by the number of elements in each tuple
    sorted_list = sorted(tuple_list, key=lambda x: len(x), reverse=True)

    list_of_lists = [list(t) for t in sorted_list]

    # Convert tuples to lists and create a dictionary with elements as keys and their corresponding tuple index as values
    for idx, tpl in enumerate(list_of_lists):
        tpl_list = list(tpl)
        for ele in tpl_list:
            tuple_dict[ele] = idx
        list_of_lists[idx] = tpl_list

    # Update the dictionary and indices based on tested elements
    for ele in tested_elements:
        if ele in tuple_dict:
            idx = tuple_dict[ele]
            if len(list_of_lists[idx]) > 1:
                list_of_lists[idx].remove(ele)
                list_of_lists.append([ele])
                tuple_dict[ele] = len(list_of_lists) - 1
                indices.append(len(list_of_lists) - 1)
            else:
                indices.append(idx)

    return list_of_lists, indices

def alpha_Sparsification(list_of_lists, data, alpha):
  list_of_tuples = [tuple(inner_list) for inner_list in list_of_lists]
  i = 0
  while len(list_of_lists[i])>1:

      Z = linkage(data.x[list_of_tuples[i],:].detach().numpy(), method='average', metric='cosine')
      threshold = alpha * np.max(Z[:, 2])
      clusters = fcluster(Z, threshold, criterion='distance')

      # Get cluster labels for each data point
      labels = clusters


      # Z = linkage(data.x[list_of_tuples[i],:], method='average', metric='euclidean')
      # threshold = alpha * np.max(Z[:, 2])
      # clusters = fcluster(Z, threshold, criterion='distance')

      # # Get cluster labels for each data point
      # labels = clusters




      # Sample tuple to split
      data_tuple = list_of_tuples[i]

      # Create a dictionary to store split data
      split_data = {}

      # Iterate through the labels and tuple elements
      for label, value in zip(labels, data_tuple):
        if label not in split_data:
          split_data[label] = []
        split_data[label].append(value)

      # Convert the dictionary to a list of tuples (if needed)
      split_tuples = [tuple(values) for key, values in split_data.items()]
      list_of_tuples.pop(i)
      j = 0
      for values in split_tuples:
        if j == 0:
          list_of_tuples.insert(0, values)
        else:
          list_of_tuples.insert(len(list_of_tuples), values)
        j+=1

      i+=1

  result_dict = {element: index for index, tup in enumerate(list_of_tuples) for element in tup}

  return list_of_tuples, result_dict

# def alpha_Sparsification(list_of_lists, data, alpha):
#     list_of_tuples = [tuple(inner_list) for inner_list in list_of_lists]
#     i = 0
#     while i<len(list_of_lists):
#         if len(list_of_lists[i])>1:
#             Z = linkage(data.x[list_of_tuples[i],:], method='average', metric='cosine')
#             threshold = alpha * np.max(Z[:, 2])
#             clusters = fcluster(Z, threshold, criterion='distance')
#             # Get cluster labels for each data point
#             labels = clusters
#             # Sample tuple to split
#             data_tuple = list_of_tuples[i]
#             # Create a dictionary to store split data
#             split_data = {}
#           # Iterate through the labels and tuple elements
#             for label, value in zip(labels, data_tuple):
#                 if label not in split_data:
#                     split_data[label] = []
#                 split_data[label].append(value)

#           # Convert the dictionary to a list of tuples (if needed)
#             split_tuples = [tuple(values) for key, values in split_data.items()]
#             list_of_tuples.pop(i)
#             j = 0
#             for values in split_tuples:
#                 if j == 0:
#                     list_of_tuples.insert(0, values)
#                 else:
#                     list_of_tuples.insert(len(list_of_tuples), values)
#                 j+=1

#         i+=1

#     result_dict = {element: index for index, tup in enumerate(list_of_tuples) for element in tup}
#     return list_of_tuples, result_dict

In [ ]:
def Sparisified_G_NX(list_of_tuples, result_dict, data):
  BCG = nx.DiGraph()

  for i in range(len(list_of_tuples)):
    BCG.add_node(i)

  for edge in G.edges():
      source, target = edge
      # if cosine_similarity(data.x[source].numpy().reshape(1, -1), data.x[target].numpy().reshape(1, -1)) > 0.5:
      BCG.add_edge(result_dict[source], result_dict[target])

  num_nodes = BCG.number_of_nodes()
  num_edges = BCG.number_of_edges()



  print("Number of nodes:", num_nodes)
  print("Number of edges:", num_edges)


  return BCG

In [ ]:
from torch_geometric.data import Data

def Sparisified_G_Alpha(BCG, data, list_of_tuples):
  num_nodes = BCG.number_of_nodes()
  new_x = torch.zeros(num_nodes,data.x.shape[1])
  new_y = torch.zeros(num_nodes)
  for i in range(len(list_of_tuples)):
    new_x[i,:] = torch.mean(data.x[list(list_of_tuples[i]),:], axis = 0)
    # new_x[i,:] = torch.sum(data.x[list(list_of_tuples[i]),:], axis = 0)
    # new_x[i, :] = torch.tensor(np.median(data.x[list(list_of_tuples[i]), :], axis=0))
    # new_x[i, :] = torch.max(data.x[list(list_of_tuples[i]), :], axis=0).values
    # new_x[i, :] = torch.min(data.x[list(list_of_tuples[i]), :], axis=0).values


    EC = list_of_tuples[i]
    L = []
    for ele in EC:
      L.append(int(data.y[ele].numpy()))

    keys = list(Counter(L).keys())
    frequencies = list(Counter(L).values())

    # Randomly select a key based on frequencies
    random_key = random.choices(keys, frequencies)[0]
    new_y[i] = int(random_key)

  print("Completed_Flag")
  edges = list(BCG.edges())

  # Create a dictionary to map node labels to integer indices
  node_mapping = {node: index for index, node in enumerate(BCG.nodes())}

  # Convert the edges to indices using the node_mapping
  new_edge_index = [[node_mapping[source], node_mapping[target]] for source, target in edges]
  new_edge_index = torch.tensor(new_edge_index, dtype=torch.long).t().contiguous()

  data_new1 = Data()
  data_new1.x = new_x
  data_new1.y = new_y.long()
  data_new1.edge_index = new_edge_index
  data_new1.num_nodes = len(new_y)

  return data_new1

In [ ]:
def Sparisified_G_Alpha_0(G, data):

  new_x = torch.zeros(len(list(G.nodes)),data.x.shape[1])
  new_y = torch.zeros(len(list(G.nodes)))
  list_itr = list(G.nodes)
  for i in range(len(list(G.nodes))):
    new_x[i,:] = data.x[list_itr[i],:]
    new_y[i] = int(data.y[list_itr[i]])

  edges = list(G.edges())

  # Create a dictionary to map node labels to integer indices
  node_mapping = {node: index for index, node in enumerate(G.nodes())}

  # Convert the edges to indices using the node_mapping
  new_edge_index = [[node_mapping[source], node_mapping[target]] for source, target in edges]
  new_edge_index = torch.tensor(new_edge_index, dtype=torch.long).t().contiguous()


  data_new1 = Data()
  data_new1.x = new_x
  data_new1.y = new_y.long()
  data_new1.edge_index = new_edge_index
  data_new1.num_nodes = len(new_y)

  return data_new1

In [ ]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
# from torch_geometric.nn import GATConv, GCNConv, GraphSAGE, BatchNorm
# import torch

# class GCN(torch.nn.Module):
#   def __init__(self):
#     super().__init__()

#     self.conv1 = GCNConv(dataset.num_node_features, 128,aggr = "sum")
#     self.bc1 = BatchNorm(128)
#     self.conv2 = GCNConv(128, 128,aggr = 'mean')
#     self.bc2 = BatchNorm(128)
#     self.conv3 = GCNConv(128, dataset.num_classes,aggr = 'mean')
#     self.bc3 = BatchNorm(dataset.num_classes)
#   def forward(self, x, edge_index, edge_weights):
#     # x: Node feature matrix
#     # edge_index: Graph connectivity matrix
#     x = self.conv1(x, edge_index, edge_weights)
#     x = torch.nn.functional.relu(x)
#     x = self.bc1(x)
#     x = self.conv2(x, edge_index, edge_weights)
#     x = torch.nn.functional.relu(x)
#     x = self.bc2(x)
#     x = self.conv3(x, edge_index, edge_weights)
#     x = torch.nn.functional.relu(x)
#     x = self.bc3(x)
#     return torch.nn.functional.log_softmax(x, dim=1)

# model = GCN()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# print("Graph Convolutional Network (GCN):")
# GCN()

# 2 layers GCN

In [ ]:
dataset.num_classes

40

In [ ]:
from torch_geometric.nn import GATConv, GCNConv, GraphSAGE, BatchNorm
import torch
from torch.nn import Module, Dropout

class GCN(torch.nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = GCNConv(800, 400)
    self.bc1 = BatchNorm(400)
    self.conv2 = GCNConv(400, 153)
    self.bc2 = BatchNorm(153)
    # self.dropout = Dropout(p=0.5)
  def forward(self, x, edge_index):
    # x: Node feature matrix
    # edge_index: Graph connectivity matrix
    x = self.conv1(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc1(x)
    # x = self.dropout(x)
    x = self.conv2(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc2(x)
    # x = self.dropout(x)
    return torch.nn.functional.log_softmax(x, dim=1)

model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
print("Graph Convolutional Network (GCN):")
GCN()

Graph Convolutional Network (GCN):


GCN(
  (conv1): GCNConv(800, 400)
  (bc1): BatchNorm(400)
  (conv2): GCNConv(400, 153)
  (bc2): BatchNorm(153)
)

# 3 Layers GCN

In [ ]:
# torch.nn.functional.log_softmax(out, dim=1).shape

torch.Size([169343, 40])

In [ ]:
# MAG240M_Syn.y = MAG240M_Syn.y.unsqueeze(1)

In [ ]:
from torch_geometric.nn import GATConv, GCNConv, GraphSAGE, BatchNorm
import torch
from torch.nn import Module, Dropout

class GCN(torch.nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = GCNConv(800, 400)
    self.bc1 = BatchNorm(400)
    self.conv2 = GCNConv(400, 200)
    self.bc2 = BatchNorm(200)
    self.conv3 = GCNConv(200, 153)
    self.bc3 = BatchNorm(153)
    # self.dropout = Dropout(p=0.5)
  def forward(self, x, edge_index):
    # x: Node feature matrix
    # edge_index: Graph connectivity matrix
    x = self.conv1(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc1(x)
    # x = self.dropout(x)
    x = self.conv2(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc2(x)
    # x = self.dropout(x)
    x = self.conv3(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc3(x)
    return torch.nn.functional.log_softmax(x, dim=1)

model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
print("Graph Convolutional Network (GCN):")
GCN()

Graph Convolutional Network (GCN):


GCN(
  (conv1): GCNConv(800, 400)
  (bc1): BatchNorm(400)
  (conv2): GCNConv(400, 200)
  (bc2): BatchNorm(200)
  (conv3): GCNConv(200, 153)
  (bc3): BatchNorm(153)
)

In [ ]:
# train the model
import time
from tqdm import tqdm
from sklearn.metrics import f1_score


# Start recording the current time
start_time = time.time()
criterion = torch.nn.BCEWithLogitsLoss()

MAG240M_Syn.x = MAG240M_Syn.x.to(dtype=torch.float32)
MAG240M_Syn.edge_index = MAG240M_Syn.edge_index.to(dtype=torch.int64)
MAG240M_Syn.y = MAG240M_Syn.y+1

losses = []
accuracies = []
num_epochs = 100

for epoch in tqdm(range(1, num_epochs + 1), desc = 'Epoch', position = 0):
  optimizer.zero_grad()
  out = model(MAG240M_Syn.x, MAG240M_Syn.edge_index)  # Ensure edge_index is of dtype int64 or long
  predictions = out
  ground_truth = MAG240M_Syn.y
  loss = torch.nn.functional.nll_loss(predictions, ground_truth)
  losses.append(loss.item())
  loss.backward()
  optimizer.step()
  # print(loss.item())
  # if (epoch+1) % 10 == 0:
  # print(epoch)
  # print(loss.item())
  print(f'Epoch {epoch} \t\t Training Loss: {loss.item()}')
  #   predicted_labels = out.detach().numpy() > 0.5
  #   true_labels = data.y.numpy()
  #   f1 = f1_score(true_labels, predicted_labels, average='micro')
  #   print("F1 Score:", f1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Training time:", elapsed_time, "seconds")


Epoch:   1%|          | 1/100 [01:47<2:57:36, 107.64s/it]

Epoch 1 		 Training Loss: 5.089441776275635


Epoch:   2%|▏         | 2/100 [03:08<2:29:37, 91.60s/it] 

Epoch 2 		 Training Loss: 10.825547218322754


Epoch:   3%|▎         | 3/100 [04:26<2:18:19, 85.56s/it]

...

Epoch 98 		 Training Loss: 5.030429840087891


Epoch:  99%|█████████▉| 99/100 [2:09:54<01:18, 78.01s/it]

Epoch 99 		 Training Loss: 5.030429363250732


Epoch: 100%|██████████| 100/100 [2:11:13<00:00, 78.73s/it]

Epoch 100 		 Training Loss: 5.030429363250732
Training time: 7877.194724321365 seconds


# 4 Layers GCN

In [ ]:
from torch_geometric.nn import GATConv, GCNConv, GraphSAGE, BatchNorm
import torch

class GCN(torch.nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = GCNConv(dataset.num_node_features, 128)
    self.bc1 = BatchNorm(128)
    self.conv2 = GCNConv(128, 128)
    self.bc2 = BatchNorm(128)
    self.conv3 = GCNConv(128, 128)
    self.bc3 = BatchNorm(128)
    self.conv4 = GCNConv(128, dataset.num_classes)
    self.bc4 = BatchNorm(dataset.num_classes)
    # self.dropout = Dropout(p=0.5)
  def forward(self, x, edge_index):
    # x: Node feature matrix
    # edge_index: Graph connectivity matrix
    x = self.conv1(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc1(x)
    x = self.conv2(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc2(x)
    x = self.conv3(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc3(x)
    x = self.conv4(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc4(x)

    return torch.nn.functional.log_softmax(x, dim=1)

model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
print("Graph Convolutional Network (GCN):")
GCN()

Graph Convolutional Network (GCN):


GCN(
  (conv1): GCNConv(128, 128)
  (bc1): BatchNorm(128)
  (conv2): GCNConv(128, 128)
  (bc2): BatchNorm(128)
  (conv3): GCNConv(128, 128)
  (bc3): BatchNorm(128)
  (conv4): GCNConv(128, 40)
  (bc4): BatchNorm(40)
)

# 2 Layers SAGE

In [ ]:
from torch_geometric.nn import GATConv, GCNConv, SAGEConv
import torch

class GraphSAGE(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = SAGEConv(800, 400)
    self.bc1 = BatchNorm(400)
    self.conv2 = SAGEConv(400, 153)
    self.bc2 = BatchNorm(153)

  def forward(self, x, edge_index):
    # x: Node feature matrix
    # edge_index: Graph connectivity matrix
    x = self.conv1(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc1(x)
    x = self.conv2(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc2(x)

    return torch.nn.functional.log_softmax(x, dim=1)

model = GraphSAGE()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
print("GraphSAGE:")
GraphSAGE()

GraphSAGE:


GraphSAGE(
  (conv1): SAGEConv(800, 400, aggr=mean)
  (bc1): BatchNorm(400)
  (conv2): SAGEConv(400, 153, aggr=mean)
  (bc2): BatchNorm(153)
)

# 3 layers SAGE

In [ ]:
from torch_geometric.nn import GATConv, GCNConv, SAGEConv
import torch

class GraphSAGE(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = SAGEConv(800, 400)
    self.bc1 = BatchNorm(400)
    self.conv2 = SAGEConv(400, 200)
    self.bc2 = BatchNorm(200)
    self.conv3 = SAGEConv(200, 153)
    self.bc3 = BatchNorm(153)

  def forward(self, x, edge_index):
    # x: Node feature matrix
    # edge_index: Graph connectivity matrix
    x = self.conv1(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc1(x)
    x = self.conv2(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc2(x)
    x = self.conv3(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc3(x)
    return torch.nn.functional.log_softmax(x, dim=1)

model = GraphSAGE()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
print("GraphSAGE:")
GraphSAGE()

GraphSAGE:


GraphSAGE(
  (conv1): SAGEConv(800, 400, aggr=mean)
  (bc1): BatchNorm(400)
  (conv2): SAGEConv(400, 200, aggr=mean)
  (bc2): BatchNorm(200)
  (conv3): SAGEConv(200, 153, aggr=mean)
  (bc3): BatchNorm(153)
)

# 4 layers SAGE

In [ ]:
from torch_geometric.nn import GATConv, GCNConv, SAGEConv
import torch
from torch.nn import Module, Dropout

class GraphSAGE(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = SAGEConv(dataset.num_node_features, 128)
    self.bc1 = BatchNorm(128)
    self.conv2 = SAGEConv(128, 128)
    self.bc2 = BatchNorm(128)
    self.conv3 = SAGEConv(128, 128)
    self.bc3 = BatchNorm(128)
    self.conv4 = SAGEConv(128, dataset.num_classes)
    self.bc4 = BatchNorm(dataset.num_classes)
    # self.dropout = Dropout(p=0.5)

  def forward(self, x, edge_index):
    # x: Node feature matrix
    # edge_index: Graph connectivity matrix
    x = self.conv1(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc1(x)
    # x = self.dropout(x)
    x = self.conv2(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc2(x)
    # x = self.dropout(x)
    x = self.conv3(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc3(x)
    x = self.conv4(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc4(x)

    return torch.nn.functional.log_softmax(x, dim=1)

model = GraphSAGE()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
print("GraphSAGE:")
GraphSAGE()

GraphSAGE:


GraphSAGE(
  (conv1): SAGEConv(128, 128, aggr=mean)
  (bc1): BatchNorm(128)
  (conv2): SAGEConv(128, 128, aggr=mean)
  (bc2): BatchNorm(128)
  (conv3): SAGEConv(128, 128, aggr=mean)
  (bc3): BatchNorm(128)
  (conv4): SAGEConv(128, 40, aggr=mean)
  (bc4): BatchNorm(40)
)

# 2 layers GAT

In [ ]:
from torch_geometric.nn import GATConv, GCNConv, SAGEConv
import torch

class GAT(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = GATConv(800, 400)
    self.bc1 = BatchNorm(400)
    self.conv2 = GATConv(400, 153)
    self.bc2 = BatchNorm(153)

  def forward(self, x, edge_index):
    # x: Node feature matrix
    # edge_index: Graph connectivity matrix
    x = self.conv1(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc1(x)
    x = self.conv2(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc2(x)
    return torch.nn.functional.log_softmax(x, dim=1)

model = GAT()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
print("GAT:")
GAT()

GAT:


GAT(
  (conv1): GATConv(800, 400, heads=1)
  (bc1): BatchNorm(400)
  (conv2): GATConv(400, 153, heads=1)
  (bc2): BatchNorm(153)
)

# 3 layers GAT

In [ ]:
from torch_geometric.nn import GATConv, GCNConv, SAGEConv
import torch

class GAT(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = GATConv(800, 400)
    self.bc1 = BatchNorm(400)
    self.conv2 = GATConv(400, 200)
    self.bc2 = BatchNorm(200)
    self.conv3 = GATConv(200, 153)
    self.bc3 = BatchNorm(153)

  def forward(self, x, edge_index):
    # x: Node feature matrix
    # edge_index: Graph connectivity matrix
    x = self.conv1(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc1(x)
    x = self.conv2(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc2(x)
    x = self.conv3(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc3(x)
    return torch.nn.functional.log_softmax(x, dim=1)

model = GAT()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
print("GAT:")
GAT()

GAT:


GAT(
  (conv1): GATConv(800, 400, heads=1)
  (bc1): BatchNorm(400)
  (conv2): GATConv(400, 200, heads=1)
  (bc2): BatchNorm(200)
  (conv3): GATConv(200, 153, heads=1)
  (bc3): BatchNorm(153)
)

# 4 layers GAT

In [ ]:
from torch_geometric.nn import GATConv, GCNConv, SAGEConv
import torch

class GAT(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = GATConv(dataset.num_node_features, 128)
    self.bc1 = BatchNorm(128)
    self.conv2 = GATConv(128, 128)
    self.bc2 = BatchNorm(128)
    self.conv3 = GATConv(128, 128)
    self.bc3 = BatchNorm(128)
    self.conv4 = GATConv(128, dataset.num_classes)
    self.bc4 = BatchNorm(dataset.num_classes)

  def forward(self, x, edge_index):
    # x: Node feature matrix
    # edge_index: Graph connectivity matrix
    x = self.conv1(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc1(x)
    # x = self.dropout(x)
    x = self.conv2(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc2(x)
    # x = self.dropout(x)
    x = self.conv3(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc3(x)
    x = self.conv4(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc4(x)

    return torch.nn.functional.log_softmax(x, dim=1)

model = GAT()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
print("GAT:")
GAT()

GAT:


GAT(
  (conv1): GATConv(128, 128, heads=1)
  (bc1): BatchNorm(128)
  (conv2): GATConv(128, 128, heads=1)
  (bc2): BatchNorm(128)
  (conv3): GATConv(128, 128, heads=1)
  (bc3): BatchNorm(128)
  (conv4): GATConv(128, 40, heads=1)
  (bc4): BatchNorm(40)
)

In [ ]:
def compute_accuracy(pred_y, y):
    return (pred_y == y).sum()

In [ ]:
# def exclude_tested_elements_from_MB(partition, tested_elements):
#   tuple_list = partition

#   # Sort the list by the number of elements in each tuple
#   sorted_list = sorted(tuple_list, key=lambda x: len(x), reverse=True)

#   indices = []
#   list_of_lists = [list(t) for t in sorted_list]

#   for ele in tested_elements:
#     id = find_tuple_index_with_integer(sorted_list, ele)
#     if len(list_of_lists[id]) > 1:
#       list_of_lists[id].remove(ele)
#       list_of_lists.append([ele])
#       indices.append(len(list_of_lists)-1)
#     else:
#       indices.append(id)

#   return list_of_lists

# def alpha_Sparsification(list_of_lists, data, alpha):
#   list_of_tuples = [tuple(inner_list) for inner_list in list_of_lists]
#   i = 0
#   while len(list_of_lists[i])>1:
#       # print(i)

#       Z = linkage(data.x[list_of_tuples[i],:], method='average', metric='cosine')
#       threshold = alpha * np.max(Z[:, 2])
#       clusters = fcluster(Z, threshold, criterion='distance')

#       # Get cluster labels for each data point
#       labels = clusters

#       # Sample tuple to split
#       data_tuple = list_of_tuples[i]

#       # Create a dictionary to store split data
#       split_data = {}

#       # Iterate through the labels and tuple elements
#       for label, value in zip(labels, data_tuple):
#         if label not in split_data:
#           split_data[label] = []
#         split_data[label].append(value)

#       # Convert the dictionary to a list of tuples (if needed)
#       split_tuples = [tuple(values) for key, values in split_data.items()]
#       list_of_tuples.pop(i)
#       j = 0
#       for values in split_tuples:
#         if j == 0:
#           list_of_tuples.insert(0, values)
#         else:
#           list_of_tuples.insert(len(list_of_tuples), values)
#         j+=1

#       i+=1

#   result_dict = {element: index for index, tup in enumerate(list_of_tuples) for element in tup}

#   return list_of_tuples, result_dict, indices

In [ ]:
similarity = cosine_similarity(data.x[0].numpy().reshape(1, -1), data.x[100].numpy().reshape(1, -1))
similarity[0][0]

0.8133455

In [ ]:
for u, v in G.edges():
    similarity = cosine_similarity(data.x[u].numpy().reshape(1, -1), data.x[v].numpy().reshape(1, -1))
    G[u][v]['weight'] = similarity[0][0]

edge_weights = [G[u][v]['weight'] for u, v in G.edges()]
# Convert edge weights to a PyTorch tensor
edge_weight_tensor = torch.tensor(edge_weights, dtype=torch.float32)
edge_weight_tensor.shape

torch.Size([221911])

In [ ]:
# for edge_list in list(G.edges):
#   similarity = cosine_similarity(data.x[edge_list[0]].numpy().reshape(1, -1), data.x[edge_list[1]].numpy().reshape(1, -1))
#   G[edge_list[0]][edge_list[1]] = similarity

# edge_weights = [G[u][v]['weight'] for u, v in BCG.edges()]
# for u, v in G.edges():
#     similarity = cosine_similarity([node_features[u]], [node_features[v]])[0][0]
#     G[u][v]['weight'] = similarity


# # Convert edge weights to a PyTorch tensor
# edge_weight_tensor = torch.tensor(edge_weights, dtype=torch.float32)
# edge_weight_tensor

# 5%

In [ ]:
# evaluate the model on test set
best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

best_model_GAT = GAT()
best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

best_model_GraphSage = GraphSAGE()
best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))


alpha_list = np.arange(0, 1.1, 0.25)
num_of_runs = 50

AVG_time_GCN = 0
AVG_time_GAT = 0
AVG_time_GraphSAGE = 0

for i in range(num_of_runs):
  start_time = time.time()
  best_model_GCN.eval()
  pred = best_model_GCN(data.x, data.edge_index).argmax(dim=1)
  # correct = compute_accuracy(pred, test_data.y)

  # End recording the current time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time
  AVG_time_GCN += elapsed_time

  if i == num_of_runs-1:
    print("GCN Inference time on original G:", AVG_time_GCN/num_of_runs, "seconds")
    correct = compute_accuracy(pred[test_mask], data.y[test_mask])
    # acc = int(correct) / len(test_data.y)
    acc = int(correct) / len(tested_elements)
    print(f'GCN Inference Accuracy: {acc:.4f}')

for j in range(num_of_runs):
  start_time = time.time()
  best_model_GAT.eval()
  pred = best_model_GAT(data.x, data.edge_index).argmax(dim=1)
  # correct = compute_accuracy(pred, test_data.y)

  # End recording the current time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time
  AVG_time_GAT += elapsed_time

  if j == num_of_runs-1:
    print("GAT Inference time on original G:", AVG_time_GAT/num_of_runs, "seconds")
    correct = compute_accuracy(pred[test_mask], data.y[test_mask])
    # acc = int(correct) / len(test_data.y)
    acc = int(correct) / len(tested_elements)
    print(f'GAT Inference Accuracy: {acc:.4f}')

for n in range(num_of_runs):
  start_time = time.time()
  best_model_GraphSage.eval()
  pred = best_model_GraphSage(data.x, data.edge_index).argmax(dim=1)
  # correct = compute_accuracy(pred, test_data.y)

  # End recording the current time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time
  AVG_time_GraphSAGE += elapsed_time

  if n == num_of_runs-1:
    print("GraphSAGE Inference time on original G:", AVG_time_GraphSAGE/num_of_runs, "seconds")
    correct = compute_accuracy(pred[test_mask], data.y[test_mask])
    # acc = int(correct) / len(test_data.y)
    acc = int(correct) / len(tested_elements)
    print(f'GraphSAGE Inference Accuracy: {acc:.4f}')


GCN Inference time on original G: 1.6490186500549315 seconds
GCN Inference Accuracy: 0.6239
GAT Inference time on original G: 1.680942120552063 seconds
GAT Inference Accuracy: 0.6256
GraphSAGE Inference time on original G: 1.4356050682067871 seconds
GraphSAGE Inference Accuracy: 0.6139


# 10%

In [ ]:
model_save_path_GraphSage

'gdrive/My Drive/GNN_Sparsification/Pre_Trained_10%_ogbn_arxiv_GraphSage.pt'

# 3

In [ ]:
MAG240M_Syn.x = MAG240M_Syn.x.to(dtype=torch.float32)
MAG240M_Syn.edge_index = MAG240M_Syn.edge_index.to(dtype=torch.int64)
MAG240M_Syn.y = MAG240M_Syn.y+1

In [ ]:
# evaluate the model on test set
def compute_accuracy(pred_y, y):
    return (pred_y == y).sum()

best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

best_model_GraphSage = GraphSAGE()
best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))

best_model_GAT = GAT()
best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

alpha_list = np.arange(0, 1.1, 0.25)
num_of_runs = 5

AVG_time_GCN = 0
AVG_time_GAT = 0
AVG_time_GraphSAGE = 0

for i in range(num_of_runs):
  start_time = time.time()
  best_model_GCN.eval()
  pred = best_model_GCN(MAG240M_Syn.x, MAG240M_Syn.edge_index).argmax(dim=1)
  # correct = compute_accuracy(pred, test_data.y)

  # End recording the current time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time
  AVG_time_GCN += elapsed_time

  if i == num_of_runs-1:
    print("GCN Inference time on original G:", AVG_time_GCN/num_of_runs, "seconds")
    correct = compute_accuracy(pred[test_mask], MAG240M_Syn.y[test_mask])
    # acc = int(correct) / len(test_data.y)
    acc = int(correct) / len(tested_elements)
    print(f'GCN Inference Accuracy: {acc:.4f}')

for j in range(num_of_runs):
  start_time = time.time()
  best_model_GAT.eval()
  pred = best_model_GAT(MAG240M_Syn.x, MAG240M_Syn.edge_index).argmax(dim=1)
  # correct = compute_accuracy(pred, test_data.y)

  # End recording the current time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time
  AVG_time_GAT += elapsed_time

  if j == num_of_runs-1:
    print("GAT Inference time on original G:", AVG_time_GAT/num_of_runs, "seconds")
    correct = compute_accuracy(pred[test_mask], MAG240M_Syn.y[test_mask])
    # acc = int(correct) / len(test_data.y)
    acc = int(correct) / len(tested_elements)
    print(f'GAT Inference Accuracy: {acc:.4f}')

for n in range(num_of_runs):
  start_time = time.time()
  best_model_GraphSage.eval()
  pred = best_model_GraphSage(MAG240M_Syn.x, MAG240M_Syn.edge_index).argmax(dim=1)
  # correct = compute_accuracy(pred, test_data.y)

  # End recording the current time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time
  AVG_time_GraphSAGE += elapsed_time

  if n == num_of_runs-1:
    print("GraphSAGE Inference time on original G:", AVG_time_GraphSAGE/num_of_runs, "seconds")
    correct = compute_accuracy(pred[test_mask], MAG240M_Syn.y[test_mask])
    # acc = int(correct) / len(test_data.y)
    acc = int(correct) / len(tested_elements)
    print(f'GraphSAGE Inference Accuracy: {acc:.4f}')


# 2

In [ ]:
model_save_path_GCN

'gdrive/My Drive/GNN_Sparsification/Pre_Trained_10%_ogbn_arxiv_GCN_4_layers.pt'

In [ ]:
# evaluate the model on test set
best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

best_model_GAT = GAT()
best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

best_model_GraphSage = GraphSAGE()
best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))


alpha_list = np.arange(0, 1.1, 0.25)
num_of_runs = 50

AVG_time_GCN = 0
AVG_time_GAT = 0
AVG_time_GraphSAGE = 0

for i in range(num_of_runs):
  start_time = time.time()
  best_model_GCN.eval()
  pred = best_model_GCN(data.x, data.edge_index).argmax(dim=1)
  # correct = compute_accuracy(pred, test_data.y)

  # End recording the current time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time
  AVG_time_GCN += elapsed_time

  if i == num_of_runs-1:
    print("GCN Inference time on original G:", AVG_time_GCN/num_of_runs, "seconds")
    correct = compute_accuracy(pred[test_mask], data.y[test_mask])
    # acc = int(correct) / len(test_data.y)
    acc = int(correct) / len(tested_elements)
    print(f'GCN Inference Accuracy: {acc:.4f}')

for j in range(num_of_runs):
  start_time = time.time()
  best_model_GAT.eval()
  pred = best_model_GAT(data.x, data.edge_index).argmax(dim=1)
  # correct = compute_accuracy(pred, test_data.y)

  # End recording the current time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time
  AVG_time_GAT += elapsed_time

  if j == num_of_runs-1:
    print("GAT Inference time on original G:", AVG_time_GAT/num_of_runs, "seconds")
    correct = compute_accuracy(pred[test_mask], data.y[test_mask])
    # acc = int(correct) / len(test_data.y)
    acc = int(correct) / len(tested_elements)
    print(f'GAT Inference Accuracy: {acc:.4f}')

for n in range(num_of_runs):
  start_time = time.time()
  best_model_GraphSage.eval()
  pred = best_model_GraphSage(data.x, data.edge_index).argmax(dim=1)
  # correct = compute_accuracy(pred, test_data.y)

  # End recording the current time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time
  AVG_time_GraphSAGE += elapsed_time

  if n == num_of_runs-1:
    print("GraphSAGE Inference time on original G:", AVG_time_GraphSAGE/num_of_runs, "seconds")
    correct = compute_accuracy(pred[test_mask], data.y[test_mask])
    # acc = int(correct) / len(test_data.y)
    acc = int(correct) / len(tested_elements)
    print(f'GraphSAGE Inference Accuracy: {acc:.4f}')


GCN Inference time on original G: 0.6927434015274048 seconds
GCN Inference Accuracy: 0.6198
GAT Inference time on original G: 0.7434472227096558 seconds
GAT Inference Accuracy: 0.6309
GraphSAGE Inference time on original G: 0.7595468711853027 seconds
GraphSAGE Inference Accuracy: 0.6166


# 4

In [ ]:
# evaluate the model on test set
best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

best_model_GAT = GAT()
best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

best_model_GraphSage = GraphSAGE()
best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))


alpha_list = np.arange(0, 1.1, 0.25)
num_of_runs = 50

AVG_time_GCN = 0
AVG_time_GAT = 0
AVG_time_GraphSAGE = 0

for i in range(num_of_runs):
  start_time = time.time()
  best_model_GCN.eval()
  pred = best_model_GCN(data.x, data.edge_index).argmax(dim=1)
  # correct = compute_accuracy(pred, test_data.y)

  # End recording the current time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time
  AVG_time_GCN += elapsed_time

  if i == num_of_runs-1:
    print("GCN Inference time on original G:", AVG_time_GCN/num_of_runs, "seconds")
    correct = compute_accuracy(pred[test_mask], data.y[test_mask])
    # acc = int(correct) / len(test_data.y)
    acc = int(correct) / len(tested_elements)
    print(f'GCN Inference Accuracy: {acc:.4f}')

for j in range(num_of_runs):
  start_time = time.time()
  best_model_GAT.eval()
  pred = best_model_GAT(data.x, data.edge_index).argmax(dim=1)
  # correct = compute_accuracy(pred, test_data.y)

  # End recording the current time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time
  AVG_time_GAT += elapsed_time

  if j == num_of_runs-1:
    print("GAT Inference time on original G:", AVG_time_GAT/num_of_runs, "seconds")
    correct = compute_accuracy(pred[test_mask], data.y[test_mask])
    # acc = int(correct) / len(test_data.y)
    acc = int(correct) / len(tested_elements)
    print(f'GAT Inference Accuracy: {acc:.4f}')

for n in range(num_of_runs):
  start_time = time.time()
  best_model_GraphSage.eval()
  pred = best_model_GraphSage(data.x, data.edge_index).argmax(dim=1)
  # correct = compute_accuracy(pred, test_data.y)

  # End recording the current time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time
  AVG_time_GraphSAGE += elapsed_time

  if n == num_of_runs-1:
    print("GraphSAGE Inference time on original G:", AVG_time_GraphSAGE/num_of_runs, "seconds")
    correct = compute_accuracy(pred[test_mask], data.y[test_mask])
    # acc = int(correct) / len(test_data.y)
    acc = int(correct) / len(tested_elements)
    print(f'GraphSAGE Inference Accuracy: {acc:.4f}')


GCN Inference time on original G: 1.626961245536804 seconds
GCN Inference Accuracy: 0.6215
GAT Inference time on original G: 1.7463735055923462 seconds
GAT Inference Accuracy: 0.6222
GraphSAGE Inference time on original G: 1.4408293294906616 seconds
GraphSAGE Inference Accuracy: 0.6023


In [ ]:
# evaluate the model on test set
best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

best_model_GAT = GAT()
best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

best_model_GraphSage = GraphSAGE()
best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))


alpha_list = np.arange(0, 1.1, 0.25)
num_of_runs = 50

AVG_time_GCN = 0
AVG_time_GAT = 0
AVG_time_GraphSAGE = 0

for i in range(num_of_runs):
  start_time = time.time()
  best_model_GCN.eval()
  pred = best_model_GCN(data.x, data.edge_index).argmax(dim=1)
  # correct = compute_accuracy(pred, test_data.y)

  # End recording the current time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time
  AVG_time_GCN += elapsed_time

  if i == num_of_runs-1:
    print("GCN Inference time on original G:", AVG_time_GCN/num_of_runs, "seconds")
    correct = compute_accuracy(pred[test_mask], data.y[test_mask])
    # acc = int(correct) / len(test_data.y)
    acc = int(correct) / len(tested_elements)
    print(f'GCN Inference Accuracy: {acc:.4f}')

for j in range(num_of_runs):
  start_time = time.time()
  best_model_GAT.eval()
  pred = best_model_GAT(data.x, data.edge_index).argmax(dim=1)
  # correct = compute_accuracy(pred, test_data.y)

  # End recording the current time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time
  AVG_time_GAT += elapsed_time

  if j == num_of_runs-1:
    print("GAT Inference time on original G:", AVG_time_GAT/num_of_runs, "seconds")
    correct = compute_accuracy(pred[test_mask], data.y[test_mask])
    # acc = int(correct) / len(test_data.y)
    acc = int(correct) / len(tested_elements)
    print(f'GAT Inference Accuracy: {acc:.4f}')

for n in range(num_of_runs):
  start_time = time.time()
  best_model_GraphSage.eval()
  pred = best_model_GraphSage(data.x, data.edge_index).argmax(dim=1)
  # correct = compute_accuracy(pred, test_data.y)

  # End recording the current time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time
  AVG_time_GraphSAGE += elapsed_time

  if n == num_of_runs-1:
    print("GraphSAGE Inference time on original G:", AVG_time_GraphSAGE/num_of_runs, "seconds")
    correct = compute_accuracy(pred[test_mask], data.y[test_mask])
    # acc = int(correct) / len(test_data.y)
    acc = int(correct) / len(tested_elements)
    print(f'GraphSAGE Inference Accuracy: {acc:.4f}')


GCN Inference time on original G: 1.572223949432373 seconds
GCN Inference Accuracy: 0.6229
GAT Inference time on original G: 1.6841576480865479 seconds
GAT Inference Accuracy: 0.6319
GraphSAGE Inference time on original G: 1.6054185438156128 seconds
GraphSAGE Inference Accuracy: 0.6173


#15%

In [ ]:
# evaluate the model on test set
best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

best_model_GAT = GAT()
best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

best_model_GraphSage = GraphSAGE()
best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))


alpha_list = np.arange(0, 1.1, 0.25)
num_of_runs = 50

AVG_time_GCN = 0
AVG_time_GAT = 0
AVG_time_GraphSAGE = 0

for i in range(num_of_runs):
  start_time = time.time()
  best_model_GCN.eval()
  pred = best_model_GCN(data.x, data.edge_index).argmax(dim=1)
  # correct = compute_accuracy(pred, test_data.y)

  # End recording the current time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time
  AVG_time_GCN += elapsed_time

  if i == num_of_runs-1:
    print("GCN Inference time on original G:", AVG_time_GCN/num_of_runs, "seconds")
    correct = compute_accuracy(pred[test_mask], data.y[test_mask])
    # acc = int(correct) / len(test_data.y)
    acc = int(correct) / len(tested_elements)
    print(f'GCN Inference Accuracy: {acc:.4f}')

for j in range(num_of_runs):
  start_time = time.time()
  best_model_GAT.eval()
  pred = best_model_GAT(data.x, data.edge_index).argmax(dim=1)
  # correct = compute_accuracy(pred, test_data.y)

  # End recording the current time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time
  AVG_time_GAT += elapsed_time

  if j == num_of_runs-1:
    print("GAT Inference time on original G:", AVG_time_GAT/num_of_runs, "seconds")
    correct = compute_accuracy(pred[test_mask], data.y[test_mask])
    # acc = int(correct) / len(test_data.y)
    acc = int(correct) / len(tested_elements)
    print(f'GAT Inference Accuracy: {acc:.4f}')

for n in range(num_of_runs):
  start_time = time.time()
  best_model_GraphSage.eval()
  pred = best_model_GraphSage(data.x, data.edge_index).argmax(dim=1)
  # correct = compute_accuracy(pred, test_data.y)

  # End recording the current time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time
  AVG_time_GraphSAGE += elapsed_time

  if n == num_of_runs-1:
    print("GraphSAGE Inference time on original G:", AVG_time_GraphSAGE/num_of_runs, "seconds")
    correct = compute_accuracy(pred[test_mask], data.y[test_mask])
    # acc = int(correct) / len(test_data.y)
    acc = int(correct) / len(tested_elements)
    print(f'GraphSAGE Inference Accuracy: {acc:.4f}')


GCN Inference time on original G: 1.510688018798828 seconds
GCN Inference Accuracy: 0.6193
GAT Inference time on original G: 1.6857985019683839 seconds
GAT Inference Accuracy: 0.6251
GraphSAGE Inference time on original G: 1.5552693271636964 seconds
GraphSAGE Inference Accuracy: 0.6008


In [ ]:
Constant_GCN = AVG_time_GCN/num_of_runs
Constant_GAT = AVG_time_GAT/num_of_runs
Constant_GraphSAGE = AVG_time_GraphSAGE/num_of_runs

In [ ]:
Constant_GCN = 39.55015344619751
Constant_GAT = 43.55015344619751
Constant_GraphSAGE = 42.57228655815125


# K-hop-Induced + SPGS

In [ ]:
# # Generate the first piece with step size 0.01 for the range [0, 0.15)
# array1 = np.arange(0, 0.01, 0.005)

# # Generate the second piece with step size 0.02 for the range [0.15, 0.3)
# array2 = np.arange(0.01, 0.2, 0.02)



# # Concatenate the two arrays
# alpha_list = np.concatenate([array1, array2])
alpha_list = np.arange(0, 0.8, 0.1)

In [ ]:
tested_elements[8000]

160958

In [ ]:
list_of_tuples[indices[8000]]

(160958,)

In [ ]:
len(new_partition)

8897

In [ ]:
alpha_list = np.arange(0, 1.05, 0.05)
alpha_list

array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ])

# 5%

In [ ]:
best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

best_model_GAT = GAT()
best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

best_model_GraphSage = GraphSAGE()
best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))

num_of_runs = 50

alpha_list = np.arange(0, 1.05, 0.05)

for alpha in alpha_list:
  print(alpha)
  if alpha == 0:
    data_new_1 = Sparisified_G_Alpha_0(G, data)
    list2 = tested_elements
    list1 = list(G.nodes)
    indices = [list1.index(x) for x in list2]

  if alpha > 0:
    if alpha == alpha_list[1]:
      list_of_lists, indices = exclude_tested_elements_from_MB(partition, tested_elements)

    list_of_tuples, result_dict = alpha_Sparsification(list_of_lists, data, alpha)
    BCG = Sparisified_G_NX(list_of_tuples, result_dict, data)
    data_new_1 = Sparisified_G_Alpha(BCG, data, list_of_tuples)

  AVG_time_GCN = 0
  AVG_time_GAT = 0
  AVG_time_GraphSAGE = 0
  print(data_new_1)
  for i in range(num_of_runs):
    start_time = time.time()
    best_model_GCN.eval()
    pred = best_model_GCN(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GCN += elapsed_time


    if i == num_of_runs-1:
      correct = compute_accuracy(pred[indices], data_new_1.y[indices])
      acc = int(correct) / len(tested_elements)
      # correct = compute_accuracy(pred, data_new_1.y)
      # acc = int(correct) / len(pred)
      print("Average of "+str(num_of_runs) + " runs GCN Inference time:", AVG_time_GCN/num_of_runs, "seconds")
      print(f'GCN Inference Speed-up: {Constant_GCN/(AVG_time_GCN/num_of_runs):.4f}')
      print(f'GCN Inference Accuracy: {acc:.4f}')

  for j in range(num_of_runs):
    AVG_time = 0
    start_time = time.time()
    best_model_GAT.eval()
    pred = best_model_GAT(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GAT += elapsed_time


    if j == num_of_runs-1:
      correct = compute_accuracy(pred[indices], data_new_1.y[indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GAT Inference time:", AVG_time_GAT/num_of_runs, "seconds")
      print(f'GAT Inference Speed-up: {Constant_GAT/(AVG_time_GAT/num_of_runs):.4f}')
      print(f'GAT Inference Accuracy: {acc:.4f}')

  for m in range(num_of_runs):
    start_time = time.time()
    best_model_GraphSage.eval()
    pred = best_model_GraphSage(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GraphSAGE += elapsed_time



    if m == num_of_runs-1:
      correct = compute_accuracy(pred[indices], data_new_1.y[indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GraphSAGE Inference time:", AVG_time_GraphSAGE/num_of_runs, "seconds")
      print(f'GraphSAGE Inference Speed-up: {Constant_GraphSAGE/(AVG_time_GraphSAGE/num_of_runs):.4f}')
      print(f'GraphSAGE Inference Accuracy: {acc:.4f}')

0.0
Data(num_nodes=60907, edge_index=[2, 104187], x=[60907, 128], node_year=[169343, 1], y=[60907])
Average of 50 runs GCN Inference time: 0.2688330268859863 seconds
GCN Inference Speed-up: 6.1340
GCN Inference Accuracy: 0.5999
Average of 50 runs GAT Inference time: 0.24636483192443848 seconds
GAT Inference Speed-up: 6.8230
GAT Inference Accuracy: 0.6045
Average of 50 runs GraphSAGE Inference time: 0.21623432636260986 seconds
GraphSAGE Inference Speed-up: 6.6391
GraphSAGE Inference Accuracy: 0.5963
0.05
Number of nodes: 46296
Number of edges: 85849
Data(num_nodes=46296, edge_index=[2, 85849], x=[46296, 128], node_year=[169343, 1], y=[46296])
Average of 50 runs GCN Inference time: 0.16262800216674805 seconds
GCN Inference Speed-up: 10.1398
GCN Inference Accuracy: 0.5800
Average of 50 runs GAT Inference time: 0.19137795448303221 seconds
GAT Inference Speed-up: 8.7834
GAT Inference Accuracy: 0.5759
Average of 50 runs GraphSAGE Inference time: 0.13248445510864257 seconds
GraphSAGE Inferenc

KeyboardInterrupt: ignored

In [ ]:
best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

best_model_GAT = GAT()
best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

best_model_GraphSage = GraphSAGE()
best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))

num_of_runs = 50

alpha_list = np.arange(0, 1.05, 0.05)

for alpha in alpha_list:
  print(alpha)
  if alpha == 0:
    data_new_1 = Sparisified_G_Alpha_0(G, data)
    list2 = tested_elements
    list1 = list(G.nodes)
    indices = [list1.index(x) for x in list2]

  if alpha > 0:
    if alpha == alpha_list[1]:
      list_of_lists, indices = exclude_tested_elements_from_MB(partition, tested_elements)

    list_of_tuples, result_dict = alpha_Sparsification(list_of_lists, data, alpha)
    BCG = Sparisified_G_NX(list_of_tuples, result_dict, data)
    data_new_1 = Sparisified_G_Alpha(BCG, data, list_of_tuples)

  AVG_time_GCN = 0
  AVG_time_GAT = 0
  AVG_time_GraphSAGE = 0
  print(data_new_1)
  for i in range(num_of_runs):
    start_time = time.time()
    best_model_GCN.eval()
    pred = best_model_GCN(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GCN += elapsed_time


    if i == num_of_runs-1:
      correct = compute_accuracy(pred[indices], data_new_1.y[indices])
      acc = int(correct) / len(tested_elements)
      # correct = compute_accuracy(pred, data_new_1.y)
      # acc = int(correct) / len(pred)
      print("Average of "+str(num_of_runs) + " runs GCN Inference time:", AVG_time_GCN/num_of_runs, "seconds")
      print(f'GCN Inference Speed-up: {Constant_GCN/(AVG_time_GCN/num_of_runs):.4f}')
      print(f'GCN Inference Accuracy: {acc:.4f}')

  for j in range(num_of_runs):
    AVG_time = 0
    start_time = time.time()
    best_model_GAT.eval()
    pred = best_model_GAT(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GAT += elapsed_time


    if j == num_of_runs-1:
      correct = compute_accuracy(pred[indices], data_new_1.y[indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GAT Inference time:", AVG_time_GAT/num_of_runs, "seconds")
      print(f'GAT Inference Speed-up: {Constant_GAT/(AVG_time_GAT/num_of_runs):.4f}')
      print(f'GAT Inference Accuracy: {acc:.4f}')

  for m in range(num_of_runs):
    start_time = time.time()
    best_model_GraphSage.eval()
    pred = best_model_GraphSage(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GraphSAGE += elapsed_time



    if m == num_of_runs-1:
      correct = compute_accuracy(pred[indices], data_new_1.y[indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GraphSAGE Inference time:", AVG_time_GraphSAGE/num_of_runs, "seconds")
      print(f'GraphSAGE Inference Speed-up: {Constant_GraphSAGE/(AVG_time_GraphSAGE/num_of_runs):.4f}')
      print(f'GraphSAGE Inference Accuracy: {acc:.4f}')

0.0
Data(num_nodes=60907, edge_index=[2, 104187], x=[60907, 128], node_year=[169343, 1], y=[60907])
Average of 50 runs GCN Inference time: 0.15171393394470215 seconds
GCN Inference Speed-up: 10.7359
GCN Inference Accuracy: 0.5999
Average of 50 runs GAT Inference time: 0.20627979755401613 seconds
GAT Inference Speed-up: 8.6431
GAT Inference Accuracy: 0.6045
Average of 50 runs GraphSAGE Inference time: 0.17893224716186523 seconds
GraphSAGE Inference Speed-up: 8.6705
GraphSAGE Inference Accuracy: 0.5963
0.05
Number of nodes: 46296
Number of edges: 85849
Data(num_nodes=46296, edge_index=[2, 85849], x=[46296, 128], node_year=[169343, 1], y=[46296])
Average of 50 runs GCN Inference time: 0.10894874095916748 seconds
GCN Inference Speed-up: 14.9500
GCN Inference Accuracy: 0.5924
Average of 50 runs GAT Inference time: 0.14259252071380615 seconds
GAT Inference Speed-up: 12.5034
GAT Inference Accuracy: 0.5802
Average of 50 runs GraphSAGE Inference time: 0.1172581958770752 seconds
GraphSAGE Infere

# 10%

# layers = 3 (Good code)

In [ ]:
def compute_accuracy(pred_y, y):
    return (pred_y == y).sum()

best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

best_model_GAT = GAT()
best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

best_model_GraphSage = GraphSAGE()
best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))

num_of_runs = 10

alpha_list = np.arange(0, 0.011, 0.02)

for alpha in alpha_list:
  print(alpha)
  if alpha == 0:
    data_new_1 = Sparisified_G_Alpha_0(G, MAG240M_Syn)
    list2 = tested_elements
    list1 = list(G.nodes)

    # indices = [list1.index(x) for x in list2]
    # Create a dictionary mapping elements to their indices in list1
    index_dict = {element: index for index, element in enumerate(list1)}

    # Use the dictionary to get the indices for elements in list2
    indices = [index_dict[x] for x in list2]

  if alpha > 0:
    if alpha == alpha_list[1]:
      list_of_lists, indices = exclude_tested_elements_from_MB(partition, tested_elements)

    list_of_tuples, result_dict = alpha_Sparsification(list_of_lists, MAG240M_Syn, alpha)
    BCG = Sparisified_G_NX(list_of_tuples, result_dict, MAG240M_Syn)
    data_new_1 = Sparisified_G_Alpha(BCG, MAG240M_Syn, list_of_tuples)

  AVG_time_GCN = 0
  AVG_time_GAT = 0
  AVG_time_GraphSAGE = 0
  print(data_new_1)
  for i in range(num_of_runs):
    start_time = time.time()
    best_model_GCN.eval()
    pred = best_model_GCN(data_new_1.x, data_new_1.edge_index).argmax(dim=1)
    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GCN += elapsed_time


    if i == num_of_runs-1:
      correct = compute_accuracy(pred[indices], data_new_1.y[indices])
      acc = int(correct) / len(tested_elements)
      # correct = compute_accuracy(pred, data_new_1.y)
      # acc = int(correct) / len(pred)
      print("Average of "+str(num_of_runs) + " runs GCN Inference time:", AVG_time_GCN/num_of_runs, "seconds")
      print(f'GCN Inference Speed-up: {Constant_GCN/(AVG_time_GCN/num_of_runs):.4f}')
      print(f'GCN Inference Accuracy: {acc:.4f}')

  for j in range(num_of_runs):
    AVG_time = 0
    start_time = time.time()
    best_model_GAT.eval()
    pred = best_model_GAT(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GAT += elapsed_time


    if j == num_of_runs-1:
      correct = compute_accuracy(pred[indices], data_new_1.y[indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GAT Inference time:", AVG_time_GAT/num_of_runs, "seconds")
      print(f'GAT Inference Speed-up: {Constant_GAT/(AVG_time_GAT/num_of_runs):.4f}')
      print(f'GAT Inference Accuracy: {acc:.4f}')

  for m in range(num_of_runs):
    start_time = time.time()
    best_model_GraphSage.eval()
    pred = best_model_GraphSage(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GraphSAGE += elapsed_time



    if m == num_of_runs-1:
      correct = compute_accuracy(pred[indices], data_new_1.y[indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GraphSAGE Inference time:", AVG_time_GraphSAGE/num_of_runs, "seconds")
      print(f'GraphSAGE Inference Speed-up: {Constant_GraphSAGE/(AVG_time_GraphSAGE/num_of_runs):.4f}')
      print(f'GraphSAGE Inference Accuracy: {acc:.4f}')

RuntimeError: Error(s) in loading state_dict for GAT:
	Missing key(s) in state_dict: "conv1.lin.weight", "conv2.lin.weight". 
	Unexpected key(s) in state_dict: "conv1.lin_src.weight", "conv1.lin_dst.weight", "conv2.lin_src.weight", "conv2.lin_dst.weight". 

In [ ]:
def compute_accuracy(pred_y, y):
    return (pred_y == y).sum()

best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

# best_model_GAT = GAT()
# best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

best_model_GraphSage = GraphSAGE()
best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))

num_of_runs = 5

alpha_list = np.arange(0, 0.8, 0.1)

for alpha in alpha_list:
  print(alpha)
  if alpha == 0:
    data_new_1 = Sparisified_G_Alpha_0(G, MAG240M_Syn)
    list2 = tested_elements
    list1 = list(G.nodes)

    # indices = [list1.index(x) for x in list2]
    # Create a dictionary mapping elements to their indices in list1
    index_dict = {element: index for index, element in enumerate(list1)}

    # Use the dictionary to get the indices for elements in list2
    indices = [index_dict[x] for x in list2]

  if alpha > 0:
    if alpha == alpha_list[1]:
      list_of_lists, indices = exclude_tested_elements_from_MB(partition, tested_elements)

    list_of_tuples, result_dict = alpha_Sparsification(list_of_lists, MAG240M_Syn, alpha)
    BCG = Sparisified_G_NX(list_of_tuples, result_dict, MAG240M_Syn)
    data_new_1 = Sparisified_G_Alpha(BCG, MAG240M_Syn, list_of_tuples)

  AVG_time_GCN = 0
  # AVG_time_GAT = 0
  AVG_time_GraphSAGE = 0
  print(data_new_1)
  for i in range(num_of_runs):
    start_time = time.time()
    best_model_GCN.eval()
    pred = best_model_GCN(data_new_1.x, data_new_1.edge_index).argmax(dim=1)
    print(i)
    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GCN += elapsed_time


    if i == num_of_runs-1:
      correct = compute_accuracy(pred[indices], data_new_1.y[indices])
      acc = int(correct) / len(tested_elements)
      # correct = compute_accuracy(pred, data_new_1.y)
      # acc = int(correct) / len(pred)
      print("Average of "+str(num_of_runs) + " runs GCN Inference time:", AVG_time_GCN/num_of_runs, "seconds")
      print(f'GCN Inference Speed-up: {Constant_GCN/(AVG_time_GCN/num_of_runs):.4f}')
      print(f'GCN Inference Accuracy: {acc:.4f}')

  # for j in range(num_of_runs):
  #   AVG_time = 0
  #   start_time = time.time()
  #   best_model_GAT.eval()
  #   pred = best_model_GAT(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

  #   # End recording the current time
  #   end_time = time.time()

  #   # Calculate the elapsed time
  #   elapsed_time = end_time - start_time
  #   AVG_time_GAT += elapsed_time


  #   if j == num_of_runs-1:
  #     correct = compute_accuracy(pred[indices], data_new_1.y[indices])
  #     acc = int(correct) / len(tested_elements)
  #     print("Average of "+str(num_of_runs) + " runs GAT Inference time:", AVG_time_GAT/num_of_runs, "seconds")
  #     print(f'GAT Inference Speed-up: {Constant_GAT/(AVG_time_GAT/num_of_runs):.4f}')
  #     print(f'GAT Inference Accuracy: {acc:.4f}')

  for m in range(num_of_runs):
    start_time = time.time()
    best_model_GraphSage.eval()
    pred = best_model_GraphSage(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GraphSAGE += elapsed_time



    if m == num_of_runs-1:
      correct = compute_accuracy(pred[indices], data_new_1.y[indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GraphSAGE Inference time:", AVG_time_GraphSAGE/num_of_runs, "seconds")
      print(f'GraphSAGE Inference Speed-up: {Constant_GraphSAGE/(AVG_time_GraphSAGE/num_of_runs):.4f}')
      print(f'GraphSAGE Inference Accuracy: {acc:.4f}')

0.0
Data(x=[260212, 800], edge_index=[2, 235220], y=[260212], num_nodes=260212)
0
1
2
3
4
Average of 5 runs GCN Inference time: 4.537987232208252 seconds
GCN Inference Speed-up: 7.2719
GCN Inference Accuracy: 0.0069
Average of 5 runs GraphSAGE Inference time: 4.080336809158325 seconds
GraphSAGE Inference Speed-up: 8.0876
GraphSAGE Inference Accuracy: 0.0073
0.1
Number of nodes: 46780
Number of edges: 151974
Completed_Flag
Data(x=[46780, 800], edge_index=[2, 151974], y=[46780], num_nodes=46780)


# layers = 3

In [ ]:
best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

best_model_GAT = GAT()
best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

best_model_GraphSage = GraphSAGE()
best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))

num_of_runs = 50

alpha_list = np.arange(0, 1.05, 0.05)

for alpha in alpha_list:
  print(alpha)
  if alpha == 0:
    data_new_1 = Sparisified_G_Alpha_0(G, data)
    list2 = tested_elements
    list1 = list(G.nodes)
    indices = [list1.index(x) for x in list2]

  if alpha > 0:
    if alpha == alpha_list[1]:
      list_of_lists, indices = exclude_tested_elements_from_MB(partition, tested_elements)

    list_of_tuples, result_dict = alpha_Sparsification(list_of_lists, data, alpha)
    BCG = Sparisified_G_NX(list_of_tuples, result_dict, data)
    data_new_1 = Sparisified_G_Alpha(BCG, data, list_of_tuples)

  AVG_time_GCN = 0
  AVG_time_GAT = 0
  AVG_time_GraphSAGE = 0
  print(data_new_1)
  for i in range(num_of_runs):
    start_time = time.time()
    best_model_GCN.eval()
    pred = best_model_GCN(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GCN += elapsed_time


    if i == num_of_runs-1:
      correct = compute_accuracy(pred[indices], data_new_1.y[indices])
      acc = int(correct) / len(tested_elements)
      # correct = compute_accuracy(pred, data_new_1.y)
      # acc = int(correct) / len(pred)
      print("Average of "+str(num_of_runs) + " runs GCN Inference time:", AVG_time_GCN/num_of_runs, "seconds")
      print(f'GCN Inference Speed-up: {Constant_GCN/(AVG_time_GCN/num_of_runs):.4f}')
      print(f'GCN Inference Accuracy: {acc:.4f}')

  for j in range(num_of_runs):
    AVG_time = 0
    start_time = time.time()
    best_model_GAT.eval()
    pred = best_model_GAT(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GAT += elapsed_time


    if j == num_of_runs-1:
      correct = compute_accuracy(pred[indices], data_new_1.y[indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GAT Inference time:", AVG_time_GAT/num_of_runs, "seconds")
      print(f'GAT Inference Speed-up: {Constant_GAT/(AVG_time_GAT/num_of_runs):.4f}')
      print(f'GAT Inference Accuracy: {acc:.4f}')

  for m in range(num_of_runs):
    start_time = time.time()
    best_model_GraphSage.eval()
    pred = best_model_GraphSage(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GraphSAGE += elapsed_time



    if m == num_of_runs-1:
      correct = compute_accuracy(pred[indices], data_new_1.y[indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GraphSAGE Inference time:", AVG_time_GraphSAGE/num_of_runs, "seconds")
      print(f'GraphSAGE Inference Speed-up: {Constant_GraphSAGE/(AVG_time_GraphSAGE/num_of_runs):.4f}')
      print(f'GraphSAGE Inference Accuracy: {acc:.4f}')

0.0
Data(num_nodes=94773, edge_index=[2, 221911], x=[94773, 128], node_year=[169343, 1], y=[94773])
Average of 50 runs GCN Inference time: 0.3033119201660156 seconds
GCN Inference Speed-up: 3.8213
GCN Inference Accuracy: 0.6040
Average of 50 runs GAT Inference time: 0.4174960660934448 seconds
GAT Inference Speed-up: 2.9805
GAT Inference Accuracy: 0.6120
Average of 50 runs GraphSAGE Inference time: 0.3416640853881836 seconds
GraphSAGE Inference Speed-up: 3.0298
GraphSAGE Inference Accuracy: 0.6005
0.05
Number of nodes: 75450
Number of edges: 194741
Data(num_nodes=75450, edge_index=[2, 194741], x=[75450, 128], node_year=[169343, 1], y=[75450])
Average of 50 runs GCN Inference time: 0.2859864282608032 seconds
GCN Inference Speed-up: 4.0528
GCN Inference Accuracy: 0.5929
Average of 50 runs GAT Inference time: 0.3241154718399048 seconds
GAT Inference Speed-up: 3.8392
GAT Inference Accuracy: 0.5937
Average of 50 runs GraphSAGE Inference time: 0.28580909729003906 seconds
GraphSAGE Inference S

# layers = 2

In [ ]:
best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

best_model_GAT = GAT()
best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

best_model_GraphSage = GraphSAGE()
best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))

num_of_runs = 50

alpha_list = np.arange(0, 1.05, 0.05)

for alpha in alpha_list:
  print(alpha)
  if alpha == 0:
    data_new_1 = Sparisified_G_Alpha_0(G, data)
    list2 = tested_elements
    list1 = list(G.nodes)
    indices = [list1.index(x) for x in list2]

  if alpha > 0:
    if alpha == alpha_list[1]:
      list_of_lists, indices = exclude_tested_elements_from_MB(partition, tested_elements)

    list_of_tuples, result_dict = alpha_Sparsification(list_of_lists, data, alpha)
    BCG = Sparisified_G_NX(list_of_tuples, result_dict, data)
    data_new_1 = Sparisified_G_Alpha(BCG, data, list_of_tuples)

  AVG_time_GCN = 0
  AVG_time_GAT = 0
  AVG_time_GraphSAGE = 0
  print(data_new_1)
  for i in range(num_of_runs):
    start_time = time.time()
    best_model_GCN.eval()
    pred = best_model_GCN(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GCN += elapsed_time


    if i == num_of_runs-1:
      correct = compute_accuracy(pred[indices], data_new_1.y[indices])
      acc = int(correct) / len(tested_elements)
      # correct = compute_accuracy(pred, data_new_1.y)
      # acc = int(correct) / len(pred)
      print("Average of "+str(num_of_runs) + " runs GCN Inference time:", AVG_time_GCN/num_of_runs, "seconds")
      print(f'GCN Inference Speed-up: {Constant_GCN/(AVG_time_GCN/num_of_runs):.4f}')
      print(f'GCN Inference Accuracy: {acc:.4f}')

  for j in range(num_of_runs):
    AVG_time = 0
    start_time = time.time()
    best_model_GAT.eval()
    pred = best_model_GAT(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GAT += elapsed_time


    if j == num_of_runs-1:
      correct = compute_accuracy(pred[indices], data_new_1.y[indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GAT Inference time:", AVG_time_GAT/num_of_runs, "seconds")
      print(f'GAT Inference Speed-up: {Constant_GAT/(AVG_time_GAT/num_of_runs):.4f}')
      print(f'GAT Inference Accuracy: {acc:.4f}')

  for m in range(num_of_runs):
    start_time = time.time()
    best_model_GraphSage.eval()
    pred = best_model_GraphSage(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GraphSAGE += elapsed_time



    if m == num_of_runs-1:
      correct = compute_accuracy(pred[indices], data_new_1.y[indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GraphSAGE Inference time:", AVG_time_GraphSAGE/num_of_runs, "seconds")
      print(f'GraphSAGE Inference Speed-up: {Constant_GraphSAGE/(AVG_time_GraphSAGE/num_of_runs):.4f}')
      print(f'GraphSAGE Inference Accuracy: {acc:.4f}')

0.0
Data(num_nodes=94773, edge_index=[2, 221911], x=[94773, 128], node_year=[169343, 1], y=[94773])
Average of 50 runs GCN Inference time: 0.20222510814666747 seconds
GCN Inference Speed-up: 1.0245
GCN Inference Accuracy: 0.6104
Average of 50 runs GAT Inference time: 0.2244918489456177 seconds
GAT Inference Speed-up: 0.0000
GAT Inference Accuracy: 0.6225
Average of 50 runs GraphSAGE Inference time: 0.22867099761962892 seconds
GraphSAGE Inference Speed-up: 0.0000
GraphSAGE Inference Accuracy: 0.6167
0.05
Number of nodes: 75450
Number of edges: 194741
Data(num_nodes=75450, edge_index=[2, 194741], x=[75450, 128], node_year=[169343, 1], y=[75450])
Average of 50 runs GCN Inference time: 0.18433625221252442 seconds
GCN Inference Speed-up: 1.1240
GCN Inference Accuracy: 0.6009
Average of 50 runs GAT Inference time: 0.19327930927276613 seconds
GAT Inference Speed-up: 0.0000
GAT Inference Accuracy: 0.6063
Average of 50 runs GraphSAGE Inference time: 0.18866573810577392 seconds
GraphSAGE Inferen

# layers = 4

In [ ]:
best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

best_model_GAT = GAT()
best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

best_model_GraphSage = GraphSAGE()
best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))

num_of_runs = 50

alpha_list = np.arange(0, 1.05, 0.05)

for alpha in alpha_list:
  print(alpha)
  if alpha == 0:
    data_new_1 = Sparisified_G_Alpha_0(G, data)
    list2 = tested_elements
    list1 = list(G.nodes)
    indices = [list1.index(x) for x in list2]

  if alpha > 0:
    if alpha == alpha_list[1]:
      list_of_lists, indices = exclude_tested_elements_from_MB(partition, tested_elements)

    list_of_tuples, result_dict = alpha_Sparsification(list_of_lists, data, alpha)
    BCG = Sparisified_G_NX(list_of_tuples, result_dict, data)
    data_new_1 = Sparisified_G_Alpha(BCG, data, list_of_tuples)

  AVG_time_GCN = 0
  AVG_time_GAT = 0
  AVG_time_GraphSAGE = 0
  print(data_new_1)
  for i in range(num_of_runs):
    start_time = time.time()
    best_model_GCN.eval()
    pred = best_model_GCN(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GCN += elapsed_time


    if i == num_of_runs-1:
      correct = compute_accuracy(pred[indices], data_new_1.y[indices])
      acc = int(correct) / len(tested_elements)
      # correct = compute_accuracy(pred, data_new_1.y)
      # acc = int(correct) / len(pred)
      print("Average of "+str(num_of_runs) + " runs GCN Inference time:", AVG_time_GCN/num_of_runs, "seconds")
      print(f'GCN Inference Speed-up: {Constant_GCN/(AVG_time_GCN/num_of_runs):.4f}')
      print(f'GCN Inference Accuracy: {acc:.4f}')

  for j in range(num_of_runs):
    AVG_time = 0
    start_time = time.time()
    best_model_GAT.eval()
    pred = best_model_GAT(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GAT += elapsed_time


    if j == num_of_runs-1:
      correct = compute_accuracy(pred[indices], data_new_1.y[indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GAT Inference time:", AVG_time_GAT/num_of_runs, "seconds")
      print(f'GAT Inference Speed-up: {Constant_GAT/(AVG_time_GAT/num_of_runs):.4f}')
      print(f'GAT Inference Accuracy: {acc:.4f}')

  for m in range(num_of_runs):
    start_time = time.time()
    best_model_GraphSage.eval()
    pred = best_model_GraphSage(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GraphSAGE += elapsed_time



    if m == num_of_runs-1:
      correct = compute_accuracy(pred[indices], data_new_1.y[indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GraphSAGE Inference time:", AVG_time_GraphSAGE/num_of_runs, "seconds")
      print(f'GraphSAGE Inference Speed-up: {Constant_GraphSAGE/(AVG_time_GraphSAGE/num_of_runs):.4f}')
      print(f'GraphSAGE Inference Accuracy: {acc:.4f}')

0.0
Data(num_nodes=94773, edge_index=[2, 221911], x=[94773, 128], node_year=[169343, 1], y=[94773])
Average of 50 runs GCN Inference time: 0.5223159456253051 seconds
GCN Inference Speed-up: 1.0045
GCN Inference Accuracy: 0.5950
Average of 50 runs GAT Inference time: 0.5959693288803101 seconds
GAT Inference Speed-up: 0.0000
GAT Inference Accuracy: 0.5934
Average of 50 runs GraphSAGE Inference time: 0.4659070873260498 seconds
GraphSAGE Inference Speed-up: 0.0000
GraphSAGE Inference Accuracy: 0.5799
0.05
Number of nodes: 75450
Number of edges: 194741
Data(num_nodes=75450, edge_index=[2, 194741], x=[75450, 128], node_year=[169343, 1], y=[75450])
Average of 50 runs GCN Inference time: 0.3927191781997681 seconds
GCN Inference Speed-up: 1.3360
GCN Inference Accuracy: 0.5826
Average of 50 runs GAT Inference time: 0.4662821388244629 seconds
GAT Inference Speed-up: 0.0000
GAT Inference Accuracy: 0.5729
Average of 50 runs GraphSAGE Inference time: 0.35180542945861815 seconds
GraphSAGE Inference S

#15%

In [ ]:
best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

best_model_GAT = GAT()
best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

best_model_GraphSage = GraphSAGE()
best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))

num_of_runs = 50

alpha_list = np.arange(0, 1.05, 0.05)

for alpha in alpha_list:
  print(alpha)
  if alpha == 0:
    data_new_1 = Sparisified_G_Alpha_0(G, data)
    list2 = tested_elements
    list1 = list(G.nodes)
    indices = [list1.index(x) for x in list2]

  if alpha > 0:
    if alpha == alpha_list[1]:
      list_of_lists, indices = exclude_tested_elements_from_MB(partition, tested_elements)

    list_of_tuples, result_dict = alpha_Sparsification(list_of_lists, data, alpha)
    BCG = Sparisified_G_NX(list_of_tuples, result_dict, data)
    data_new_1 = Sparisified_G_Alpha(BCG, data, list_of_tuples)

  AVG_time_GCN = 0
  AVG_time_GAT = 0
  AVG_time_GraphSAGE = 0
  print(data_new_1)
  for i in range(num_of_runs):
    start_time = time.time()
    best_model_GCN.eval()
    pred = best_model_GCN(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GCN += elapsed_time


    if i == num_of_runs-1:
      correct = compute_accuracy(pred[indices], data_new_1.y[indices])
      acc = int(correct) / len(tested_elements)
      # correct = compute_accuracy(pred, data_new_1.y)
      # acc = int(correct) / len(pred)
      print("Average of "+str(num_of_runs) + " runs GCN Inference time:", AVG_time_GCN/num_of_runs, "seconds")
      print(f'GCN Inference Speed-up: {Constant_GCN/(AVG_time_GCN/num_of_runs):.4f}')
      print(f'GCN Inference Accuracy: {acc:.4f}')

  for j in range(num_of_runs):
    AVG_time = 0
    start_time = time.time()
    best_model_GAT.eval()
    pred = best_model_GAT(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GAT += elapsed_time


    if j == num_of_runs-1:
      correct = compute_accuracy(pred[indices], data_new_1.y[indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GAT Inference time:", AVG_time_GAT/num_of_runs, "seconds")
      print(f'GAT Inference Speed-up: {Constant_GAT/(AVG_time_GAT/num_of_runs):.4f}')
      print(f'GAT Inference Accuracy: {acc:.4f}')

  for m in range(num_of_runs):
    start_time = time.time()
    best_model_GraphSage.eval()
    pred = best_model_GraphSage(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GraphSAGE += elapsed_time



    if m == num_of_runs-1:
      correct = compute_accuracy(pred[indices], data_new_1.y[indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GraphSAGE Inference time:", AVG_time_GraphSAGE/num_of_runs, "seconds")
      print(f'GraphSAGE Inference Speed-up: {Constant_GraphSAGE/(AVG_time_GraphSAGE/num_of_runs):.4f}')
      print(f'GraphSAGE Inference Accuracy: {acc:.4f}')

0.0
Data(num_nodes=115255, edge_index=[2, 340335], x=[115255, 128], node_year=[169343, 1], y=[115255])
Average of 50 runs GCN Inference time: 0.6674854803085327 seconds
GCN Inference Speed-up: 2.2633
GCN Inference Accuracy: 0.6016
Average of 50 runs GAT Inference time: 0.7859863328933716 seconds
GAT Inference Speed-up: 2.1448
GAT Inference Accuracy: 0.6076
Average of 50 runs GraphSAGE Inference time: 0.6949956607818604 seconds
GraphSAGE Inference Speed-up: 2.2378
GraphSAGE Inference Accuracy: 0.5870
0.05
Number of nodes: 98308
Number of edges: 314265
Data(num_nodes=98308, edge_index=[2, 314265], x=[98308, 128], node_year=[169343, 1], y=[98308])
Average of 50 runs GCN Inference time: 0.6056373929977417 seconds
GCN Inference Speed-up: 2.4944
GCN Inference Accuracy: 0.5941
Average of 50 runs GAT Inference time: 0.6664122819900513 seconds
GAT Inference Speed-up: 2.5297
GAT Inference Accuracy: 0.5939
Average of 50 runs GraphSAGE Inference time: 0.5863761949539185 seconds
GraphSAGE Inference

# K-hop-Induced + K-spanner

In [ ]:
# Generate a list of keys where values belong to specified_values
new_indices = [key for key, value in mapping.items() if value in tested_elements]

# Print the result
print("Keys with specified values:", new_indices)

Keys with specified values: [0, 3, 11, 13, 15, 16, 17, 22, 24, 27, 30, 44, 46, 51, 53, 61, 62, 66, 67, 68, 75, 90, 94, 97, 105, 107, 116, 124, 130, 143, 148, 153, 163, 165, 171, 178, 185, 187, 194, 202, 208, 216, 217, 219, 237, 239, 245, 252, 253, 256, 268, 284, 293, 301, 313, 315, 317, 321, 331, 334, 345, 356, 364, 371, 374, 395, 401, 402, 404, 408, 412, 418, 419, 420, 422, 423, 424, 426, 428, 431, 433, 447, 451, 452, 457, 460, 463, 466, 469, 471, 509, 510, 512, 513, 519, 525, 529, 531, 532, 571, 576, 577, 582, 583, 588, 591, 598, 600, 628, 631, 636, 638, 648, 650, 656, 657, 661, 668, 671, 682, 685, 697, 698, 705, 709, 714, 716, 718, 719, 724, 732, 736, 742, 746, 751, 753, 756, 760, 762, 777, 785, 789, 793, 798, 801, 802, 833, 840, 851, 881, 887, 896, 904, 912, 918, 928, 933, 940, 941, 950, 953, 962, 965, 968, 969, 975, 981, 995, 997, 1000, 1001, 1010, 1019, 1027, 1038, 1039, 1044, 1048, 1063, 1064, 1066, 1073, 1077, 1078, 1090, 1097, 1099, 1104, 1106, 1117, 1123, 1135, 1140, 1141, 11

# 5%

In [ ]:
best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

best_model_GAT = GAT()
best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

best_model_GraphSage = GraphSAGE()
best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))
num_of_runs = 50


K_list = [2, 3, 4, 5, 6, 7, 8, 9, 10]
for k in K_list:
  new_x = torch.zeros(len(list(G.nodes)),data.x.shape[1])
  new_y = torch.zeros(len(list(G.nodes)))

  for i in range(len(list(G.nodes))):
    new_x[i,:] = data.x[mapping[i],:]
    new_y[i] = int(data.y[mapping[i]])

  data_new_1 = copy.copy(data)
  data_new_1.x = new_x
  data_new_1.y = new_y.long()
  data_new_1.num_nodes = len(new_y)
  loaded_tensor = torch.load(f"gdrive/My Drive/GNN_Sparsification/Baselines/K_spanner_INDUCED/5%/{k:.0f}_spanner.pt")
  data_new_1.edge_index = loaded_tensor

  print(k)
  print(data_new_1)


  AVG_time_GCN = 0
  AVG_time_GAT = 0
  AVG_time_GraphSAGE = 0

  for i in range(num_of_runs):
    start_time = time.time()
    best_model_GCN.eval()
    pred = best_model_GCN(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GCN += elapsed_time


    if i == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GCN Inference time:", AVG_time_GCN/num_of_runs, "seconds")
      print(f'GCN Inference Speed-up: {Constant_GCN/(AVG_time_GCN/num_of_runs):.4f}')
      print(f'GCN Inference Accuracy: {acc:.4f}')

  for j in range(num_of_runs):
    AVG_time = 0
    start_time = time.time()
    best_model_GAT.eval()
    pred = best_model_GAT(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GAT += elapsed_time


    if j == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GAT Inference time:", AVG_time_GAT/num_of_runs, "seconds")
      print(f'GAT Inference Speed-up: {Constant_GAT/(AVG_time_GAT/num_of_runs):.4f}')
      print(f'GAT Inference Accuracy: {acc:.4f}')

  for m in range(num_of_runs):
    start_time = time.time()
    best_model_GraphSage.eval()
    pred = best_model_GraphSage(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GraphSAGE += elapsed_time



    if m == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GraphSAGE Inference time:", AVG_time_GraphSAGE/num_of_runs, "seconds")
      print(f'GraphSAGE Inference Speed-up: {Constant_GraphSAGE/(AVG_time_GraphSAGE/num_of_runs):.4f}')
      print(f'GraphSAGE Inference Accuracy: {acc:.4f}')

  epsilon+=0.1


2
Data(num_nodes=60907, edge_index=[2, 101806], x=[60907, 128], node_year=[169343, 1], y=[60907])
Average of 50 runs GCN Inference time: 0.2120392656326294 seconds
GCN Inference Speed-up: 7.6815
GCN Inference Accuracy: 0.3497
Average of 50 runs GAT Inference time: 0.20593105316162108 seconds
GAT Inference Speed-up: 8.6577
GAT Inference Accuracy: 0.3354
Average of 50 runs GraphSAGE Inference time: 0.2569160079956055 seconds
GraphSAGE Inference Speed-up: 6.0386
GraphSAGE Inference Accuracy: 0.4007
3
Data(num_nodes=60907, edge_index=[2, 99990], x=[60907, 128], node_year=[169343, 1], y=[60907])
Average of 50 runs GCN Inference time: 0.2125902557373047 seconds
GCN Inference Speed-up: 7.6616
GCN Inference Accuracy: 0.3495
Average of 50 runs GAT Inference time: 0.2652449035644531 seconds
GAT Inference Speed-up: 6.7217
GAT Inference Accuracy: 0.3357
Average of 50 runs GraphSAGE Inference time: 0.20508988857269286 seconds
GraphSAGE Inference Speed-up: 7.5646
GraphSAGE Inference Accuracy: 0.4009

KeyboardInterrupt: ignored

# 10%

In [ ]:
best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

best_model_GAT = GAT()
best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

best_model_GraphSage = GraphSAGE()
best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))
num_of_runs = 50


K_list = [2, 3, 4, 5, 6, 7, 8, 9, 10]
for k in K_list:
  new_x = torch.zeros(len(list(G.nodes)),data.x.shape[1])
  new_y = torch.zeros(len(list(G.nodes)))

  for i in range(len(list(G.nodes))):
    new_x[i,:] = data.x[mapping[i],:]
    new_y[i] = int(data.y[mapping[i]])

  data_new_1 = copy.copy(data)
  data_new_1.x = new_x
  data_new_1.y = new_y.long()
  data_new_1.num_nodes = len(new_y)
  loaded_tensor = torch.load(f"gdrive/My Drive/GNN_Sparsification/Baselines/K_spanner_INDUCED/10%/{k:.0f}_spanner.pt")
  data_new_1.edge_index = loaded_tensor

  print(k)
  print(data_new_1)


  AVG_time_GCN = 0
  AVG_time_GAT = 0
  AVG_time_GraphSAGE = 0

  for i in range(num_of_runs):
    start_time = time.time()
    best_model_GCN.eval()
    pred = best_model_GCN(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GCN += elapsed_time


    if i == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GCN Inference time:", AVG_time_GCN/num_of_runs, "seconds")
      print(f'GCN Inference Speed-up: {Constant_GCN/(AVG_time_GCN/num_of_runs):.4f}')
      print(f'GCN Inference Accuracy: {acc:.4f}')

  for j in range(num_of_runs):
    AVG_time = 0
    start_time = time.time()
    best_model_GAT.eval()
    pred = best_model_GAT(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GAT += elapsed_time


    if j == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GAT Inference time:", AVG_time_GAT/num_of_runs, "seconds")
      print(f'GAT Inference Speed-up: {Constant_GAT/(AVG_time_GAT/num_of_runs):.4f}')
      print(f'GAT Inference Accuracy: {acc:.4f}')

  for m in range(num_of_runs):
    start_time = time.time()
    best_model_GraphSage.eval()
    pred = best_model_GraphSage(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GraphSAGE += elapsed_time



    if m == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GraphSAGE Inference time:", AVG_time_GraphSAGE/num_of_runs, "seconds")
      print(f'GraphSAGE Inference Speed-up: {Constant_GraphSAGE/(AVG_time_GraphSAGE/num_of_runs):.4f}')
      print(f'GraphSAGE Inference Accuracy: {acc:.4f}')

  epsilon+=0.1


2
Data(num_nodes=94773, edge_index=[2, 208418], x=[94773, 128], node_year=[169343, 1], y=[94773])
Average of 50 runs GCN Inference time: 0.47451082706451414 seconds
GCN Inference Speed-up: 3.3134
GCN Inference Accuracy: 0.3400
Average of 50 runs GAT Inference time: 0.5152646160125732 seconds
GAT Inference Speed-up: 3.2685
GAT Inference Accuracy: 0.3438
Average of 50 runs GraphSAGE Inference time: 0.45415514945983887 seconds
GraphSAGE Inference Speed-up: 3.5350
GraphSAGE Inference Accuracy: 0.4096
3
Data(num_nodes=94773, edge_index=[2, 200053], x=[94773, 128], node_year=[169343, 1], y=[94773])
Average of 50 runs GCN Inference time: 0.45684433937072755 seconds
GCN Inference Speed-up: 3.4415
GCN Inference Accuracy: 0.3401
Average of 50 runs GAT Inference time: 0.537569055557251 seconds
GAT Inference Speed-up: 3.1329
GAT Inference Accuracy: 0.3436
Average of 50 runs GraphSAGE Inference time: 0.46977504253387453 seconds
GraphSAGE Inference Speed-up: 3.4174
GraphSAGE Inference Accuracy: 0.40

#15%

In [ ]:
best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

best_model_GAT = GAT()
best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

best_model_GraphSage = GraphSAGE()
best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))
num_of_runs = 50


K_list = [2, 3, 4, 5, 6, 7, 8, 9, 10]
for k in K_list:
  new_x = torch.zeros(len(list(G.nodes)),data.x.shape[1])
  new_y = torch.zeros(len(list(G.nodes)))

  for i in range(len(list(G.nodes))):
    new_x[i,:] = data.x[mapping[i],:]
    new_y[i] = int(data.y[mapping[i]])

  data_new_1 = copy.copy(data)
  data_new_1.x = new_x
  data_new_1.y = new_y.long()
  data_new_1.num_nodes = len(new_y)
  loaded_tensor = torch.load(f"gdrive/My Drive/GNN_Sparsification/Baselines/K_spanner_INDUCED/15%/{k:.0f}_spanner.pt")
  data_new_1.edge_index = loaded_tensor

  print(k)
  print(data_new_1)


  AVG_time_GCN = 0
  AVG_time_GAT = 0
  AVG_time_GraphSAGE = 0

  for i in range(num_of_runs):
    start_time = time.time()
    best_model_GCN.eval()
    pred = best_model_GCN(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GCN += elapsed_time


    if i == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GCN Inference time:", AVG_time_GCN/num_of_runs, "seconds")
      print(f'GCN Inference Speed-up: {Constant_GCN/(AVG_time_GCN/num_of_runs):.4f}')
      print(f'GCN Inference Accuracy: {acc:.4f}')

  for j in range(num_of_runs):
    AVG_time = 0
    start_time = time.time()
    best_model_GAT.eval()
    pred = best_model_GAT(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GAT += elapsed_time


    if j == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GAT Inference time:", AVG_time_GAT/num_of_runs, "seconds")
      print(f'GAT Inference Speed-up: {Constant_GAT/(AVG_time_GAT/num_of_runs):.4f}')
      print(f'GAT Inference Accuracy: {acc:.4f}')

  for m in range(num_of_runs):
    start_time = time.time()
    best_model_GraphSage.eval()
    pred = best_model_GraphSage(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GraphSAGE += elapsed_time



    if m == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GraphSAGE Inference time:", AVG_time_GraphSAGE/num_of_runs, "seconds")
      print(f'GraphSAGE Inference Speed-up: {Constant_GraphSAGE/(AVG_time_GraphSAGE/num_of_runs):.4f}')
      print(f'GraphSAGE Inference Accuracy: {acc:.4f}')

  epsilon+=0.1


2
Data(num_nodes=115255, edge_index=[2, 310796], x=[115255, 128], node_year=[169343, 1], y=[115255])
Average of 50 runs GCN Inference time: 0.6375626659393311 seconds
GCN Inference Speed-up: 2.3695
GCN Inference Accuracy: 0.3312
Average of 50 runs GAT Inference time: 0.7395412158966065 seconds
GAT Inference Speed-up: 2.2795
GAT Inference Accuracy: 0.3268
Average of 50 runs GraphSAGE Inference time: 0.6365594005584717 seconds
GraphSAGE Inference Speed-up: 2.4432
GraphSAGE Inference Accuracy: 0.3917
3
Data(num_nodes=115255, edge_index=[2, 295133], x=[115255, 128], node_year=[169343, 1], y=[115255])
Average of 50 runs GCN Inference time: 0.6081552362442016 seconds
GCN Inference Speed-up: 2.4841
GCN Inference Accuracy: 0.3310
Average of 50 runs GAT Inference time: 0.7020256090164184 seconds
GAT Inference Speed-up: 2.4013
GAT Inference Accuracy: 0.3269
Average of 50 runs GraphSAGE Inference time: 0.5914565134048462 seconds
GraphSAGE Inference Speed-up: 2.6296
GraphSAGE Inference Accuracy: 0

# K-hop-Induced + Dspar

# 5%

In [ ]:
best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

best_model_GAT = GAT()
best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

best_model_GraphSage = GraphSAGE()
best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))
num_of_runs = 50


epsilon = 0.35
while epsilon <= 1.55:
  new_x = torch.zeros(len(list(G.nodes)),data.x.shape[1])
  new_y = torch.zeros(len(list(G.nodes)))

  for i in range(len(list(G.nodes))):
    new_x[i,:] = data.x[mapping[i],:]
    new_y[i] = int(data.y[mapping[i]])

  data_new_1 = copy.copy(data)
  data_new_1.x = new_x
  data_new_1.y = new_y.long()
  data_new_1.num_nodes = len(new_y)
  loaded_tensor = torch.load(f"gdrive/My Drive/GNN_Sparsification/Baselines/Dspar_INDUCED/5%/Arxiv/5%_dir_induced_arxiv_sparsified_{epsilon:.2f}.pt")
  data_new_1.edge_index = loaded_tensor[1]

  print(epsilon)
  print(data_new_1)


  AVG_time_GCN = 0
  AVG_time_GAT = 0
  AVG_time_GraphSAGE = 0

  for i in range(num_of_runs):
    start_time = time.time()
    best_model_GCN.eval()
    pred = best_model_GCN(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GCN += elapsed_time


    if i == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GCN Inference time:", AVG_time_GCN/num_of_runs, "seconds")
      print(f'GCN Inference Speed-up: {Constant_GCN/(AVG_time_GCN/num_of_runs):.4f}')
      print(f'GCN Inference Accuracy: {acc:.4f}')

  for j in range(num_of_runs):
    AVG_time = 0
    start_time = time.time()
    best_model_GAT.eval()
    pred = best_model_GAT(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GAT += elapsed_time


    if j == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GAT Inference time:", AVG_time_GAT/num_of_runs, "seconds")
      print(f'GAT Inference Speed-up: {Constant_GAT/(AVG_time_GAT/num_of_runs):.4f}')
      print(f'GAT Inference Accuracy: {acc:.4f}')

  for m in range(num_of_runs):
    start_time = time.time()
    best_model_GraphSage.eval()
    pred = best_model_GraphSage(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GraphSAGE += elapsed_time



    if m == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GraphSAGE Inference time:", AVG_time_GraphSAGE/num_of_runs, "seconds")
      print(f'GraphSAGE Inference Speed-up: {Constant_GraphSAGE/(AVG_time_GraphSAGE/num_of_runs):.4f}')
      print(f'GraphSAGE Inference Accuracy: {acc:.4f}')

  epsilon+=0.1


0.35
Data(num_nodes=60907, edge_index=[2, 57748], x=[60907, 128], node_year=[169343, 1], y=[60907])
Average of 50 runs GCN Inference time: 0.19810327053070068 seconds
GCN Inference Speed-up: 8.2219
GCN Inference Accuracy: 0.5913
Average of 50 runs GAT Inference time: 0.2100733423233032 seconds
GAT Inference Speed-up: 8.4870
GAT Inference Accuracy: 0.5933
Average of 50 runs GraphSAGE Inference time: 0.20807376861572266 seconds
GraphSAGE Inference Speed-up: 7.4561
GraphSAGE Inference Accuracy: 0.5873
0.44999999999999996
Data(num_nodes=60907, edge_index=[2, 57740], x=[60907, 128], node_year=[169343, 1], y=[60907])
Average of 50 runs GCN Inference time: 0.16660559177398682 seconds
GCN Inference Speed-up: 9.7763
GCN Inference Accuracy: 0.5913
Average of 50 runs GAT Inference time: 0.2074846124649048 seconds
GAT Inference Speed-up: 8.5929
GAT Inference Accuracy: 0.5933
Average of 50 runs GraphSAGE Inference time: 0.17004549503326416 seconds
GraphSAGE Inference Speed-up: 9.1236
GraphSAGE Infe

# 10%

# layers = 3

In [ ]:
best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

best_model_GAT = GAT()
best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

best_model_GraphSage = GraphSAGE()
best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))
num_of_runs = 50


epsilon = 0.35
while epsilon <= 1.55:
  new_x = torch.zeros(len(list(G.nodes)),data.x.shape[1])
  new_y = torch.zeros(len(list(G.nodes)))

  for i in range(len(list(G.nodes))):
    new_x[i,:] = data.x[mapping[i],:]
    new_y[i] = int(data.y[mapping[i]])

  data_new_1 = copy.copy(data)
  data_new_1.x = new_x
  data_new_1.y = new_y.long()
  data_new_1.num_nodes = len(new_y)
  loaded_tensor = torch.load(f"gdrive/My Drive/GNN_Sparsification/Baselines/Dspar_INDUCED/10%/Arxiv/10%_dir_induced_arxiv_sparsified_{epsilon:.2f}.pt")
  data_new_1.edge_index = loaded_tensor[1]

  print(epsilon)
  print(data_new_1)


  AVG_time_GCN = 0
  AVG_time_GAT = 0
  AVG_time_GraphSAGE = 0

  for i in range(num_of_runs):
    start_time = time.time()
    best_model_GCN.eval()
    pred = best_model_GCN(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GCN += elapsed_time


    if i == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GCN Inference time:", AVG_time_GCN/num_of_runs, "seconds")
      print(f'GCN Inference Speed-up: {Constant_GCN/(AVG_time_GCN/num_of_runs):.4f}')
      print(f'GCN Inference Accuracy: {acc:.4f}')

  for j in range(num_of_runs):
    AVG_time = 0
    start_time = time.time()
    best_model_GAT.eval()
    pred = best_model_GAT(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GAT += elapsed_time


    if j == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GAT Inference time:", AVG_time_GAT/num_of_runs, "seconds")
      print(f'GAT Inference Speed-up: {Constant_GAT/(AVG_time_GAT/num_of_runs):.4f}')
      print(f'GAT Inference Accuracy: {acc:.4f}')

  for m in range(num_of_runs):
    start_time = time.time()
    best_model_GraphSage.eval()
    pred = best_model_GraphSage(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GraphSAGE += elapsed_time



    if m == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GraphSAGE Inference time:", AVG_time_GraphSAGE/num_of_runs, "seconds")
      print(f'GraphSAGE Inference Speed-up: {Constant_GraphSAGE/(AVG_time_GraphSAGE/num_of_runs):.4f}')
      print(f'GraphSAGE Inference Accuracy: {acc:.4f}')

  epsilon+=0.1


0.35
Data(num_nodes=94773, edge_index=[2, 115055], x=[94773, 128], node_year=[169343, 1], y=[94773])
Average of 50 runs GCN Inference time: 0.22958401203155518 seconds
GCN Inference Speed-up: 5.0485
GCN Inference Accuracy: 0.5914
Average of 50 runs GAT Inference time: 0.2997851753234863 seconds
GAT Inference Speed-up: 4.1508
GAT Inference Accuracy: 0.5962
Average of 50 runs GraphSAGE Inference time: 0.26263946056365967 seconds
GraphSAGE Inference Speed-up: 3.9414
GraphSAGE Inference Accuracy: 0.5871
0.44999999999999996
Data(num_nodes=94773, edge_index=[2, 114982], x=[94773, 128], node_year=[169343, 1], y=[94773])
Average of 50 runs GCN Inference time: 0.20363768100738525 seconds
GCN Inference Speed-up: 5.6918
GCN Inference Accuracy: 0.5913
Average of 50 runs GAT Inference time: 0.25981419563293456 seconds
GAT Inference Speed-up: 4.7894
GAT Inference Accuracy: 0.5962
Average of 50 runs GraphSAGE Inference time: 0.21280792236328125 seconds
GraphSAGE Inference Speed-up: 4.8643
GraphSAGE I

# layers = 2

In [ ]:
best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

best_model_GAT = GAT()
best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

best_model_GraphSage = GraphSAGE()
best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))
num_of_runs = 50


epsilon = 0.35
while epsilon <= 1.55:
  new_x = torch.zeros(len(list(G.nodes)),data.x.shape[1])
  new_y = torch.zeros(len(list(G.nodes)))

  for i in range(len(list(G.nodes))):
    new_x[i,:] = data.x[mapping[i],:]
    new_y[i] = int(data.y[mapping[i]])

  data_new_1 = copy.copy(data)
  data_new_1.x = new_x
  data_new_1.y = new_y.long()
  data_new_1.num_nodes = len(new_y)
  loaded_tensor = torch.load(f"gdrive/My Drive/GNN_Sparsification/Baselines/Dspar_INDUCED/10%/Arxiv/10%_dir_induced_arxiv_sparsified_{epsilon:.2f}.pt")
  data_new_1.edge_index = loaded_tensor[1]

  print(epsilon)
  print(data_new_1)


  AVG_time_GCN = 0
  AVG_time_GAT = 0
  AVG_time_GraphSAGE = 0

  for i in range(num_of_runs):
    start_time = time.time()
    best_model_GCN.eval()
    pred = best_model_GCN(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GCN += elapsed_time


    if i == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GCN Inference time:", AVG_time_GCN/num_of_runs, "seconds")
      print(f'GCN Inference Speed-up: {Constant_GCN/(AVG_time_GCN/num_of_runs):.4f}')
      print(f'GCN Inference Accuracy: {acc:.4f}')

  for j in range(num_of_runs):
    AVG_time = 0
    start_time = time.time()
    best_model_GAT.eval()
    pred = best_model_GAT(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GAT += elapsed_time


    if j == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GAT Inference time:", AVG_time_GAT/num_of_runs, "seconds")
      print(f'GAT Inference Speed-up: {Constant_GAT/(AVG_time_GAT/num_of_runs):.4f}')
      print(f'GAT Inference Accuracy: {acc:.4f}')

  for m in range(num_of_runs):
    start_time = time.time()
    best_model_GraphSage.eval()
    pred = best_model_GraphSage(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GraphSAGE += elapsed_time



    if m == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GraphSAGE Inference time:", AVG_time_GraphSAGE/num_of_runs, "seconds")
      print(f'GraphSAGE Inference Speed-up: {Constant_GraphSAGE/(AVG_time_GraphSAGE/num_of_runs):.4f}')
      print(f'GraphSAGE Inference Accuracy: {acc:.4f}')

  epsilon+=0.1


0.35
Data(num_nodes=94773, edge_index=[2, 115055], x=[94773, 128], node_year=[169343, 1], y=[94773])
Average of 50 runs GCN Inference time: 0.19705509185791015 seconds
GCN Inference Speed-up: 1.0514
GCN Inference Accuracy: 0.5978
Average of 50 runs GAT Inference time: 0.22536667346954345 seconds
GAT Inference Speed-up: 0.0000
GAT Inference Accuracy: 0.6108
Average of 50 runs GraphSAGE Inference time: 0.17713069438934326 seconds
GraphSAGE Inference Speed-up: 0.0000
GraphSAGE Inference Accuracy: 0.5995
0.44999999999999996
Data(num_nodes=94773, edge_index=[2, 114982], x=[94773, 128], node_year=[169343, 1], y=[94773])
Average of 50 runs GCN Inference time: 0.16811712265014647 seconds
GCN Inference Speed-up: 1.2324
GCN Inference Accuracy: 0.5977
Average of 50 runs GAT Inference time: 0.18532583713531495 seconds
GAT Inference Speed-up: 0.0000
GAT Inference Accuracy: 0.6107
Average of 50 runs GraphSAGE Inference time: 0.15117013931274415 seconds
GraphSAGE Inference Speed-up: 0.0000
GraphSAGE 

# 4

In [ ]:
best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

best_model_GAT = GAT()
best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

best_model_GraphSage = GraphSAGE()
best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))
num_of_runs = 50


epsilon = 0.35
while epsilon <= 1.55:
  new_x = torch.zeros(len(list(G.nodes)),data.x.shape[1])
  new_y = torch.zeros(len(list(G.nodes)))

  for i in range(len(list(G.nodes))):
    new_x[i,:] = data.x[mapping[i],:]
    new_y[i] = int(data.y[mapping[i]])

  data_new_1 = copy.copy(data)
  data_new_1.x = new_x
  data_new_1.y = new_y.long()
  data_new_1.num_nodes = len(new_y)
  loaded_tensor = torch.load(f"gdrive/My Drive/GNN_Sparsification/Baselines/Dspar_INDUCED/10%/Arxiv/10%_dir_induced_arxiv_sparsified_{epsilon:.2f}.pt")
  data_new_1.edge_index = loaded_tensor[1]

  print(epsilon)
  print(data_new_1)


  AVG_time_GCN = 0
  AVG_time_GAT = 0
  AVG_time_GraphSAGE = 0

  for i in range(num_of_runs):
    start_time = time.time()
    best_model_GCN.eval()
    pred = best_model_GCN(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GCN += elapsed_time


    if i == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GCN Inference time:", AVG_time_GCN/num_of_runs, "seconds")
      print(f'GCN Inference Speed-up: {Constant_GCN/(AVG_time_GCN/num_of_runs):.4f}')
      print(f'GCN Inference Accuracy: {acc:.4f}')

  for j in range(num_of_runs):
    AVG_time = 0
    start_time = time.time()
    best_model_GAT.eval()
    pred = best_model_GAT(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GAT += elapsed_time


    if j == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GAT Inference time:", AVG_time_GAT/num_of_runs, "seconds")
      print(f'GAT Inference Speed-up: {Constant_GAT/(AVG_time_GAT/num_of_runs):.4f}')
      print(f'GAT Inference Accuracy: {acc:.4f}')

  for m in range(num_of_runs):
    start_time = time.time()
    best_model_GraphSage.eval()
    pred = best_model_GraphSage(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GraphSAGE += elapsed_time



    if m == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GraphSAGE Inference time:", AVG_time_GraphSAGE/num_of_runs, "seconds")
      print(f'GraphSAGE Inference Speed-up: {Constant_GraphSAGE/(AVG_time_GraphSAGE/num_of_runs):.4f}')
      print(f'GraphSAGE Inference Accuracy: {acc:.4f}')

  epsilon+=0.1


0.35
Data(num_nodes=94773, edge_index=[2, 115055], x=[94773, 128], node_year=[169343, 1], y=[94773])
Average of 50 runs GCN Inference time: 0.2950981616973877 seconds
GCN Inference Speed-up: 1.7779
GCN Inference Accuracy: 0.5820
Average of 50 runs GAT Inference time: 0.37792932987213135 seconds
GAT Inference Speed-up: 0.0000
GAT Inference Accuracy: 0.5779
Average of 50 runs GraphSAGE Inference time: 0.2675423574447632 seconds
GraphSAGE Inference Speed-up: 0.0000
GraphSAGE Inference Accuracy: 0.5635
0.44999999999999996
Data(num_nodes=94773, edge_index=[2, 114982], x=[94773, 128], node_year=[169343, 1], y=[94773])
Average of 50 runs GCN Inference time: 0.2715766477584839 seconds
GCN Inference Speed-up: 1.9319
GCN Inference Accuracy: 0.5820
Average of 50 runs GAT Inference time: 0.37454150676727294 seconds
GAT Inference Speed-up: 0.0000
GAT Inference Accuracy: 0.5779
Average of 50 runs GraphSAGE Inference time: 0.26241332054138183 seconds
GraphSAGE Inference Speed-up: 0.0000
GraphSAGE Inf

#15%

In [ ]:
best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

best_model_GAT = GAT()
best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

best_model_GraphSage = GraphSAGE()
best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))
num_of_runs = 50


epsilon = 0.35
while epsilon <= 1.55:
  new_x = torch.zeros(len(list(G.nodes)),data.x.shape[1])
  new_y = torch.zeros(len(list(G.nodes)))

  for i in range(len(list(G.nodes))):
    new_x[i,:] = data.x[mapping[i],:]
    new_y[i] = int(data.y[mapping[i]])

  data_new_1 = copy.copy(data)
  data_new_1.x = new_x
  data_new_1.y = new_y.long()
  data_new_1.num_nodes = len(new_y)
  loaded_tensor = torch.load(f"gdrive/My Drive/GNN_Sparsification/Baselines/Dspar_INDUCED/15%/Arxiv/15%_dir_induced_arxiv_sparsified_{epsilon:.2f}.pt")
  data_new_1.edge_index = loaded_tensor[1]

  print(epsilon)
  print(data_new_1)


  AVG_time_GCN = 0
  AVG_time_GAT = 0
  AVG_time_GraphSAGE = 0

  for i in range(num_of_runs):
    start_time = time.time()
    best_model_GCN.eval()
    pred = best_model_GCN(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GCN += elapsed_time


    if i == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GCN Inference time:", AVG_time_GCN/num_of_runs, "seconds")
      print(f'GCN Inference Speed-up: {Constant_GCN/(AVG_time_GCN/num_of_runs):.4f}')
      print(f'GCN Inference Accuracy: {acc:.4f}')

  for j in range(num_of_runs):
    AVG_time = 0
    start_time = time.time()
    best_model_GAT.eval()
    pred = best_model_GAT(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GAT += elapsed_time


    if j == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GAT Inference time:", AVG_time_GAT/num_of_runs, "seconds")
      print(f'GAT Inference Speed-up: {Constant_GAT/(AVG_time_GAT/num_of_runs):.4f}')
      print(f'GAT Inference Accuracy: {acc:.4f}')

  for m in range(num_of_runs):
    start_time = time.time()
    best_model_GraphSage.eval()
    pred = best_model_GraphSage(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GraphSAGE += elapsed_time



    if m == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GraphSAGE Inference time:", AVG_time_GraphSAGE/num_of_runs, "seconds")
      print(f'GraphSAGE Inference Speed-up: {Constant_GraphSAGE/(AVG_time_GraphSAGE/num_of_runs):.4f}')
      print(f'GraphSAGE Inference Accuracy: {acc:.4f}')

  epsilon+=0.1


0.35
Data(num_nodes=115255, edge_index=[2, 170857], x=[115255, 128], node_year=[169343, 1], y=[115255])
Average of 50 runs GCN Inference time: 0.6231170129776001 seconds
GCN Inference Speed-up: 2.4244
GCN Inference Accuracy: 0.5813
Average of 50 runs GAT Inference time: 0.7054054737091064 seconds
GAT Inference Speed-up: 2.3898
GAT Inference Accuracy: 0.5866
Average of 50 runs GraphSAGE Inference time: 0.5437310838699341 seconds
GraphSAGE Inference Speed-up: 2.8604
GraphSAGE Inference Accuracy: 0.5677
0.44999999999999996
Data(num_nodes=115255, edge_index=[2, 170494], x=[115255, 128], node_year=[169343, 1], y=[115255])
Average of 50 runs GCN Inference time: 0.5150676488876342 seconds
GCN Inference Speed-up: 2.9330
GCN Inference Accuracy: 0.5813
Average of 50 runs GAT Inference time: 0.5968287134170532 seconds
GAT Inference Speed-up: 2.8246
GAT Inference Accuracy: 0.5866
Average of 50 runs GraphSAGE Inference time: 0.5382809829711914 seconds
GraphSAGE Inference Speed-up: 2.8893
GraphSAGE 

In [ ]:
best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

best_model_GAT = GAT()
best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

best_model_GraphSage = GraphSAGE()
best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))
num_of_runs = 100


epsilon = 0.35
while epsilon <= 1.55:
  new_x = torch.zeros(len(list(G.nodes)),data.x.shape[1])
  new_y = torch.zeros(len(list(G.nodes)))

  for i in range(len(list(G.nodes))):
    new_x[i,:] = data.x[mapping[i],:]
    new_y[i] = int(data.y[mapping[i]])

  data_new_1 = copy.copy(data)
  data_new_1.x = new_x
  data_new_1.y = new_y.long()
  data_new_1.num_nodes = len(new_y)
  loaded_tensor = torch.load(f"gdrive/My Drive/GNN_Sparsification/Baselines/Dspar_INDUCED/5%/Arxiv/5%_dir_induced_arxiv_sparsified_{epsilon:.2f}.pt")
  data_new_1.edge_index = loaded_tensor[1]

  print(epsilon)


  AVG_time_GCN = 0
  AVG_time_GAT = 0
  AVG_time_GraphSAGE = 0

  for i in range(num_of_runs):
    start_time = time.time()
    best_model_GCN.eval()
    pred = best_model_GCN(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GCN += elapsed_time


    if i == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GCN Inference time:", AVG_time_GCN/num_of_runs, "seconds")
      print(f'GCN Inference Speed-up: {Constant_GCN/(AVG_time_GCN/num_of_runs):.4f}')
      print(f'GCN Inference Accuracy: {acc:.4f}')

  for j in range(num_of_runs):
    AVG_time = 0
    start_time = time.time()
    best_model_GAT.eval()
    pred = best_model_GAT(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GAT += elapsed_time


    if j == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GAT Inference time:", AVG_time_GAT/num_of_runs, "seconds")
      print(f'GAT Inference Speed-up: {Constant_GAT/(AVG_time_GAT/num_of_runs):.4f}')
      print(f'GAT Inference Accuracy: {acc:.4f}')

  for m in range(num_of_runs):
    start_time = time.time()
    best_model_GraphSage.eval()
    pred = best_model_GraphSage(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GraphSAGE += elapsed_time



    if m == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GraphSAGE Inference time:", AVG_time_GraphSAGE/num_of_runs, "seconds")
      print(f'GraphSAGE Inference Speed-up: {Constant_GraphSAGE/(AVG_time_GraphSAGE/num_of_runs):.4f}')
      print(f'GraphSAGE Inference Accuracy: {acc:.4f}')

  epsilon+=0.2


0.35
Average of 100 runs GCN Inference time: 0.10185845136642456 seconds
GCN Inference Speed-up: 12.2042
GCN Inference Accuracy: 0.5913
Average of 100 runs GAT Inference time: 0.12392525911331177 seconds
GAT Inference Speed-up: 11.1616
GAT Inference Accuracy: 0.5933
Average of 100 runs GraphSAGE Inference time: 0.10914520025253296 seconds
GraphSAGE Inference Speed-up: 11.3681
GraphSAGE Inference Accuracy: 0.5873
0.55
Average of 100 runs GCN Inference time: 0.08539613008499146 seconds
GCN Inference Speed-up: 14.5569
GCN Inference Accuracy: 0.5910
Average of 100 runs GAT Inference time: 0.12259222269058227 seconds
GAT Inference Speed-up: 11.2830
GAT Inference Accuracy: 0.5931
Average of 100 runs GraphSAGE Inference time: 0.10775201797485351 seconds
GraphSAGE Inference Speed-up: 11.5150
GraphSAGE Inference Accuracy: 0.5870
0.75
Average of 100 runs GCN Inference time: 0.08717181205749512 seconds
GCN Inference Speed-up: 14.2604
GCN Inference Accuracy: 0.5887
Average of 100 runs GAT Inferenc

In [ ]:
best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

best_model_GAT = GAT()
best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

best_model_GraphSage = GraphSAGE()
best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))
num_of_runs = 100


epsilon = 0.35
while epsilon <= 1.55:
  new_x = torch.zeros(len(list(G.nodes)),data.x.shape[1])
  new_y = torch.zeros(len(list(G.nodes)))

  for i in range(len(list(G.nodes))):
    new_x[i,:] = data.x[mapping[i],:]
    new_y[i] = int(data.y[mapping[i]])

  data_new_1 = copy.copy(data)
  data_new_1.x = new_x
  data_new_1.y = new_y.long()
  data_new_1.num_nodes = len(new_y)
  loaded_tensor = torch.load(f"gdrive/My Drive/GNN_Sparsification/Baselines/Dspar_INDUCED/5%/5%_dir_induced_arxiv_sparsified_{epsilon:.2f}.pt")
  data_new_1.edge_index = loaded_tensor[1]

  print(epsilon)


  AVG_time_GCN = 0
  AVG_time_GAT = 0
  AVG_time_GraphSAGE = 0

  for i in range(num_of_runs):
    start_time = time.time()
    best_model_GCN.eval()
    pred = best_model_GCN(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GCN += elapsed_time


    if i == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GCN Inference time:", AVG_time_GCN/num_of_runs, "seconds")
      print(f'GCN Inference Speed-up: {Constant_GCN/(AVG_time_GCN/num_of_runs):.4f}')
      print(f'GCN Inference Accuracy: {acc:.4f}')

  for j in range(num_of_runs):
    AVG_time = 0
    start_time = time.time()
    best_model_GAT.eval()
    pred = best_model_GAT(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GAT += elapsed_time


    if j == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GAT Inference time:", AVG_time_GAT/num_of_runs, "seconds")
      print(f'GAT Inference Speed-up: {Constant_GAT/(AVG_time_GAT/num_of_runs):.4f}')
      print(f'GAT Inference Accuracy: {acc:.4f}')

  for m in range(num_of_runs):
    start_time = time.time()
    best_model_GraphSage.eval()
    pred = best_model_GraphSage(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GraphSAGE += elapsed_time



    if m == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GraphSAGE Inference time:", AVG_time_GraphSAGE/num_of_runs, "seconds")
      print(f'GraphSAGE Inference Speed-up: {Constant_GraphSAGE/(AVG_time_GraphSAGE/num_of_runs):.4f}')
      print(f'GraphSAGE Inference Accuracy: {acc:.4f}')

  epsilon+=0.2


0.35
Average of 100 runs GCN Inference time: 0.1054825735092163 seconds
GCN Inference Speed-up: 11.6881
GCN Inference Accuracy: 0.5913
Average of 100 runs GAT Inference time: 0.15996865034103394 seconds
GAT Inference Speed-up: 8.9148
GAT Inference Accuracy: 0.5933
Average of 100 runs GraphSAGE Inference time: 0.12928671836853028 seconds
GraphSAGE Inference Speed-up: 10.0053
GraphSAGE Inference Accuracy: 0.5873
0.55
Average of 100 runs GCN Inference time: 0.13353700160980225 seconds
GCN Inference Speed-up: 9.2326
GCN Inference Accuracy: 0.5910
Average of 100 runs GAT Inference time: 0.15879172801971436 seconds
GAT Inference Speed-up: 8.9808
GAT Inference Accuracy: 0.5931
Average of 100 runs GraphSAGE Inference time: 0.10846877813339234 seconds
GraphSAGE Inference Speed-up: 11.9256
GraphSAGE Inference Accuracy: 0.5870
0.75
Average of 100 runs GCN Inference time: 0.11737947702407837 seconds
GCN Inference Speed-up: 10.5034
GCN Inference Accuracy: 0.5887
Average of 100 runs GAT Inference ti

In [ ]:
best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

best_model_GAT = GAT()
best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

best_model_GraphSage = GraphSAGE()
best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))
num_of_runs = 100


epsilon = 0.35
while epsilon <= 1.55:
  new_x = torch.zeros(len(list(G.nodes)),data.x.shape[1])
  new_y = torch.zeros(len(list(G.nodes)))

  for i in range(len(list(G.nodes))):
    new_x[i,:] = data.x[mapping[i],:]
    new_y[i] = int(data.y[mapping[i]])

  data_new_1 = copy.copy(data)
  data_new_1.x = new_x
  data_new_1.y = new_y.long()
  data_new_1.num_nodes = len(new_y)
  loaded_tensor = torch.load(f"gdrive/My Drive/GNN_Sparsification/Baselines/Dspar_INDUCED/10%/10%_dir_induced_arxiv_sparsified_{epsilon:.2f}.pt")
  data_new_1.edge_index = loaded_tensor[1]

  print(epsilon)


  AVG_time_GCN = 0
  AVG_time_GAT = 0
  AVG_time_GraphSAGE = 0

  for i in range(num_of_runs):
    start_time = time.time()
    best_model_GCN.eval()
    pred = best_model_GCN(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GCN += elapsed_time


    if i == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GCN Inference time:", AVG_time_GCN/num_of_runs, "seconds")
      print(f'GCN Inference Speed-up: {Constant_GCN/(AVG_time_GCN/num_of_runs):.4f}')
      print(f'GCN Inference Accuracy: {acc:.4f}')

  for j in range(num_of_runs):
    AVG_time = 0
    start_time = time.time()
    best_model_GAT.eval()
    pred = best_model_GAT(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GAT += elapsed_time


    if j == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GAT Inference time:", AVG_time_GAT/num_of_runs, "seconds")
      print(f'GAT Inference Speed-up: {Constant_GAT/(AVG_time_GAT/num_of_runs):.4f}')
      print(f'GAT Inference Accuracy: {acc:.4f}')

  for m in range(num_of_runs):
    start_time = time.time()
    best_model_GraphSage.eval()
    pred = best_model_GraphSage(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GraphSAGE += elapsed_time



    if m == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GraphSAGE Inference time:", AVG_time_GraphSAGE/num_of_runs, "seconds")
      print(f'GraphSAGE Inference Speed-up: {Constant_GraphSAGE/(AVG_time_GraphSAGE/num_of_runs):.4f}')
      print(f'GraphSAGE Inference Accuracy: {acc:.4f}')

  epsilon+=0.2


0.35
Average of 100 runs GCN Inference time: 0.268235399723053 seconds
GCN Inference Speed-up: 3.5008
GCN Inference Accuracy: 0.5631
Average of 100 runs GAT Inference time: 0.322331395149231 seconds
GAT Inference Speed-up: 3.3957
GAT Inference Accuracy: 0.5623
Average of 100 runs GraphSAGE Inference time: 0.18289923429489136 seconds
GraphSAGE Inference Speed-up: 3.2712
GraphSAGE Inference Accuracy: 0.5751
0.55
Average of 100 runs GCN Inference time: 0.2711269783973694 seconds
GCN Inference Speed-up: 3.4635
GCN Inference Accuracy: 0.5624
Average of 100 runs GAT Inference time: 0.32890404224395753 seconds
GAT Inference Speed-up: 3.3278
GAT Inference Accuracy: 0.5619
Average of 100 runs GraphSAGE Inference time: 0.18586703300476073 seconds
GraphSAGE Inference Speed-up: 3.2190
GraphSAGE Inference Accuracy: 0.5750
0.75
Average of 100 runs GCN Inference time: 0.27358327150344847 seconds
GCN Inference Speed-up: 3.4324
GCN Inference Accuracy: 0.5596
Average of 100 runs GAT Inference time: 0.33